In [1]:
import os
import numpy as np
import pandas as pd
from math import ceil
from collections import deque
from itertools import combinations
import math
import sys
global maxdays
current_directory = os.getcwd()
print("Current working directory:", current_directory)


Current working directory: /Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project


In [2]:
%cd /Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project


/Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project


In [3]:
"""
Enter filepath of instance and desired name for the solution file
"""
directory = "/Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project"
filename = 'CO_Case2412.txt'
output_file_name = "CO_Case2412_sol.txt"


"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}

print("Machines:")
print(machines)

print("\nLocations:")
print(locations)

print("\nRequests:")
print(requests)

print("\nTechnicians:")
print(technicians)


print(DAYS)
print(TRUCK_CAPACITY)
print(TRUCK_MAX_DISTANCE)

print(TRUCK_DISTANCE_COST)
print(TRUCK_DAY_COST)
print(TRUCK_COST)
print(TECHNICIAN_DISTANCE_COST)
print(TECHNICIAN_DAY_COST)
print(TECHNICIAN_COST)

Machines:
[[  1   6 155]
 [  2   3 421]
 [  3   7 593]]

Locations:
[[  1 675 563]
 [  2 742 453]
 [  3 325 514]
 [  4 595 438]
 [  5 583 353]
 [  6 316 466]
 [  7 608 353]
 [  8 326 473]
 [  9 533 437]
 [ 10 689 690]
 [ 11 551 677]
 [ 12 571 620]
 [ 13 411 373]
 [ 14 742 455]
 [ 15 543 360]
 [ 16 237 349]]

Requests:
[[ 1  8  1  2  3  1]
 [ 2  3  1  4  3  2]
 [ 3 11  1  4  2  2]
 [ 4  4  1  4  3  1]
 [ 5 12  2  3  2  1]
 [ 6 12  2  3  2  2]
 [ 7 11  2  4  3  1]
 [ 8  2  2  5  3  2]
 [ 9  2  2  3  3  2]
 [10 10  2  4  2  2]
 [11  4  3  4  3  1]
 [12 12  3  6  1  1]
 [13 15  3  5  3  1]
 [14  9  3  5  2  2]
 [15 14  3  4  3  1]
 [16 15  4  7  1  2]
 [17  3  4  5  2  1]
 [18  4  4  5  3  1]
 [19  2  4  6  3  2]
 [20  8  4  7  3  2]
 [21  6  4  7  3  1]
 [22  5  5  7  3  2]
 [23  4  5  8  2  3]
 [24 13  5  6  2  3]
 [25  5  6  7  3  2]
 [26  4  6  7  2  1]
 [27  3  6  8  2  2]
 [28 12  6  8  2  1]
 [29 14  6  9  2  1]
 [30  6  6  9  1  1]]

Technicians:
[[  1   7 212   5   0   1   0]
 [  

In [4]:
def nearest_neighbor(point, unvisited_points, locations):
    """
    Find the nearest neighbor of a given point among a list of unvisited points.

    Args:
    - point: The point for which the nearest neighbor is to be found.
    - unvisited_points: List of unvisited points.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - nearest_point: The nearest neighbor of the given point.
    """
    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_point in unvisited_points:
        candidate_point_id = int(candidate_point)
        if candidate_point_id != int(point):
            candidate_coordinates = get_location_coordinates(candidate_point_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = candidate_point_id
    return nearest_point

def get_location_coordinates(location_id, locations):
    """
    Get the coordinates of a location based on its ID.

    Args:
    - location_id: The ID of the location.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - coordinates: Tuple (x, y) representing the coordinates of the location.
    """
    for location in locations:
        if location[0] == location_id:
            return (location[1], location[2])
    return None



def calculate_distance(coordinates1, coordinates2):
    """
    Calculate the Euclidean distance between two points given their coordinates
    and round up to the nearest integer using the ceil function.

    Args:
    - coordinates1: Tuple (x1, y1) representing the coordinates of the first point.
    - coordinates2: Tuple (x2, y2) representing the coordinates of the second point.

    Returns:
    - distance: The Euclidean distance between the two points, rounded up to the nearest integer.
    """
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return ceil(distance)




def calculate_distance_between_requests(request_id1, request_id2, requests, locations):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - distance: The Euclidean distance between the locations of the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None



def get_location_details(requests, locations, request_id):
    """
    Get the location details (ID and coordinates) corresponding to the given request ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id: The request ID for which the location details are to be retrieved.

    Returns:
    - location_details: A tuple containing the location ID and its corresponding coordinates [(location_id, x_coordinate, y_coordinate)]
    - None if the request ID is not found.
    """
    for request in requests:
        if request[0] == request_id:  # Check if request ID matches
            location_id = request[1]
            for location in locations:
                if location[0] == location_id:  # Find location details by matching location ID
                    return (location_id, location[1], location[2])  # Return location details
    return None  # Return None if request ID not found

def calculate_request_distance(requests, locations, request_id1, request_id2):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.

    Returns:
    - distance: The Euclidean distance between the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        print(f"Location corresponding to Request ID {request_id1}: ID={location1[0]}, Coordinates=({location1[1]}, {location1[2]})")
        print(f"Location corresponding to Request ID {request_id2}: ID={location2[0]}, Coordinates=({location2[1]}, {location2[2]})")
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

# Example usage:
request_id1_input = 3
request_id2_input = 5

distance = calculate_request_distance(requests, locations, request_id1_input, request_id2_input)
if distance is not None:
    print(f"Distance between Request ID {request_id1_input} and Request ID {request_id2_input}: {distance:.2f}")
else:
    print("One or both request IDs not found.")
    
    
    
def get_requests_for_location(requests, location_id):
    """
    Get the list of request IDs corresponding to the given location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - location_id: The location ID for which the request IDs are to be retrieved.

    Returns:
    - request_ids: A list of request IDs corresponding to the given location ID.
    """
    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids

unique_location_ids = set(req[1] for req in requests)

# Iterate over unique location IDs
for location_id in unique_location_ids:
    # Get request IDs for the current location ID
    request_ids_for_location = get_requests_for_location(requests, location_id)
    # Print location ID and corresponding request IDs
    print(f"Requests belonging to Location ID {location_id}: {request_ids_for_location}")
    
    
def group_requests_by_location(requests):
    """
    Group requests by their location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]

    Returns:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    location_groups = {}
    for request in requests:
        request_id, location_id = request[0], request[1]
        if location_id not in location_groups:
            location_groups[location_id] = []
        location_groups[location_id].append(request_id)
    return location_groups

def print_location_groups(location_groups):
    """
    Print the location groups.

    Args:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    for location_id, request_ids in location_groups.items():
        print(f"Location ID {location_id}: {request_ids}")

location_groups = group_requests_by_location(requests)
print_location_groups(location_groups)



def calculate_request_weight(request_id, requests, machines):
    request = [req for req in requests if req[0] == request_id][0]
    machine_kind_id = request[4]
    num_requested_machines = request[5]
    machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
    return machine_size * num_requested_machines



def check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines):
    depot_loc_ID = depot[0]
    truck_loads = []
    current_truck_load = []
    current_truck_weight = 0
    
    for request_id, location_id, _, _, machine_kind_id, num_requested_machines in requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if current_truck_weight + request_weight > TRUCK_CAPACITY:
            if current_truck_load:
                truck_loads.append(current_truck_load)
            current_truck_load = [(request_id, request_weight)]
            current_truck_weight = request_weight
        else:
            current_truck_load.append((request_id, request_weight))
            current_truck_weight += request_weight
    

# Example usage
check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines)









Location corresponding to Request ID 3: ID=11, Coordinates=(551, 677)
Location corresponding to Request ID 5: ID=12, Coordinates=(571, 620)
Distance between Request ID 3 and Request ID 5: 61.00
Requests belonging to Location ID 2: [8, 9, 19]
Requests belonging to Location ID 3: [2, 17, 27]
Requests belonging to Location ID 4: [4, 11, 18, 23, 26]
Requests belonging to Location ID 5: [22, 25]
Requests belonging to Location ID 6: [21, 30]
Requests belonging to Location ID 8: [1, 20]
Requests belonging to Location ID 9: [14]
Requests belonging to Location ID 10: [10]
Requests belonging to Location ID 11: [3, 7]
Requests belonging to Location ID 12: [5, 6, 12, 28]
Requests belonging to Location ID 13: [24]
Requests belonging to Location ID 14: [15, 29]
Requests belonging to Location ID 15: [13, 16]
Location ID 8: [1, 20]
Location ID 3: [2, 17, 27]
Location ID 11: [3, 7]
Location ID 4: [4, 11, 18, 23, 26]
Location ID 12: [5, 6, 12, 28]
Location ID 2: [8, 9, 19]
Location ID 10: [10]
Location 

In [5]:
def get_machine_type_from_request(requests):
    """
    Extract the machine type from each request and store it in a dictionary.

    Args:
    - requests: List of requests where each request is a tuple.

    Returns:
    - machine_types: Dictionary where keys are request IDs and values are machine types.
    """
    machine_types = {}
    for request in requests:
        request_id = request[0]
        machine_type = request[4]  # Assuming the machine type is the 5th entry
        machine_types[request_id] = machine_type

    return machine_types

machine_types = get_machine_type_from_request(requests)

# Print the machine types for each request
print("Machine Types for Each Request:")
for request_id, machine_type in machine_types.items():
    print(f"Request {request_id}: {machine_type}")

machine_types

Machine Types for Each Request:
Request 1: 3
Request 2: 3
Request 3: 2
Request 4: 3
Request 5: 2
Request 6: 2
Request 7: 3
Request 8: 3
Request 9: 3
Request 10: 2
Request 11: 3
Request 12: 1
Request 13: 3
Request 14: 2
Request 15: 3
Request 16: 1
Request 17: 2
Request 18: 3
Request 19: 3
Request 20: 3
Request 21: 3
Request 22: 3
Request 23: 2
Request 24: 2
Request 25: 3
Request 26: 2
Request 27: 2
Request 28: 2
Request 29: 2
Request 30: 1


{1: 3,
 2: 3,
 3: 2,
 4: 3,
 5: 2,
 6: 2,
 7: 3,
 8: 3,
 9: 3,
 10: 2,
 11: 3,
 12: 1,
 13: 3,
 14: 2,
 15: 3,
 16: 1,
 17: 2,
 18: 3,
 19: 3,
 20: 3,
 21: 3,
 22: 3,
 23: 2,
 24: 2,
 25: 3,
 26: 2,
 27: 2,
 28: 2,
 29: 2,
 30: 1}

In [6]:
def get_penalty_cost_from_machine(machines):
    """
    Extract the penalty cost from each request.

    Args:
    - requests: List of requests where each request is a tuple.

    Returns:
    - penalty_cost: Dictionary where keys are request IDs and values are the penalty costs.
    """
    penalty_cost = {}
    for mach in machines:
        penalty_cost[mach[0]] = mach[2]  # Assuming the penalty cost is the 3rd entry in each request tuple
    return penalty_cost

penalty_costs_machines = get_penalty_cost_from_machine(machines)
print(penalty_costs_machines)

{1: 155, 2: 421, 3: 593}


In [7]:
# eerst maak method met distance tussen depot location (loc 1) en elke andere locatie

# dan maak method met distance tussen opeenvolgende requests, store dit voor alle opeenvolgende requests in array/dictionary

# maak method die kijkt van depot naar 2 requests, naar depot met afstand

# dan maak method met truck routes met dagen zoals hieronder, maar check ook of de truck geen route rijd zoals hierboven in method

In [8]:
def calculate_depot_to_request_distance(locations, depot_loc_ID, request_id):
    """
    Calculate the distance between the depot and a specific request based on their corresponding location coordinates.

    Args:
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - depot_loc_ID: The location ID of the depot.
    - request_id: The request ID of the request.

    Returns:
    - distance: The Euclidean distance between the depot and the request.
    - None if either the depot location ID or request ID is not found.
    """
    depot_coordinates = get_location_coordinates(depot_loc_ID, locations)
    request_location_id = [request[1] for request in requests if request[0] == request_id]
    if request_location_id:
        request_coordinates = get_location_coordinates(request_location_id[0], locations)
        distance = calculate_distance(depot_coordinates, request_coordinates)
        return distance
    else:
        return None

# Calculate the distances between the depot and each request
depot_loc_ID = 1  # Assuming the depot location ID is always 1
depot_to_requests = {}
for request_id in range(1, len(requests) + 1):
    distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id)
    if distance is not None:
        depot_to_requests[request_id] = distance

# Now you can access the distance for a specific request ID
print(depot_to_requests)
print(depot_to_requests[1])

{1: 361, 2: 354, 3: 169, 4: 149, 5: 119, 6: 119, 7: 169, 8: 129, 9: 129, 10: 128, 11: 149, 12: 119, 13: 243, 14: 190, 15: 128, 16: 243, 17: 354, 18: 149, 19: 129, 20: 361, 21: 372, 22: 230, 23: 149, 24: 326, 25: 230, 26: 149, 27: 354, 28: 119, 29: 128, 30: 372}
361


In [9]:
def distance_between_consecutive_requests(locations, requests, request_id1, request_id2):
    """
    Calculate the distance between two consecutive requests based on their corresponding location coordinates.

    Args:
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.

    Returns:
    - distance: The Euclidean distance between the two consecutive requests.
    - None if either request ID is not found or if the requests are not consecutive.
    """
    location_id1 = None
    location_id2 = None

    # Find the location IDs corresponding to the given request IDs
    for request in requests:
        if request[0] == request_id1:
            location_id1 = request[1]
        elif request[0] == request_id2:
            location_id2 = request[1]

    # If both location IDs are found, calculate the distance
    if location_id1 is not None and location_id2 is not None:
        coordinates1 = get_location_coordinates(location_id1, locations)
        coordinates2 = get_location_coordinates(location_id2, locations)
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

def calculate_consecutive_request_distances(locations, requests):
    """
    Calculate and store the distances between all pairs of consecutive requests.

    Args:
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]

    Returns:
    - consecutive_distances: Dictionary storing the distances between consecutive requests.
    """
    consecutive_distances = {}

    # Iterate through the requests to calculate distances between consecutive pairs
    for i in range(len(requests) - 1):
        request_id1 = requests[i][0]
        request_id2 = requests[i + 1][0]
        distance = distance_between_consecutive_requests(locations, requests, request_id1, request_id2)
        if distance is not None:
            consecutive_distances[(request_id1, request_id2)] = distance

    return consecutive_distances

# Calculate and store the distances between consecutive requests
consecutive_distances = calculate_consecutive_request_distances(locations, requests)
print(consecutive_distances)

# Print the distance between requests 2 and 3

print(consecutive_distances[(1, 2)])


{(1, 2): 42, (2, 3): 279, (3, 4): 244, (4, 5): 184, (5, 6): 0, (6, 7): 61, (7, 8): 295, (8, 9): 0, (9, 10): 243, (10, 11): 269, (11, 12): 184, (12, 13): 262, (13, 14): 78, (14, 15): 210, (15, 16): 221, (16, 17): 267, (17, 18): 281, (18, 19): 148, (19, 20): 417, (20, 21): 13, (21, 22): 290, (22, 23): 86, (23, 24): 196, (24, 25): 174, (25, 26): 86, (26, 27): 281, (27, 28): 268, (28, 29): 238, (29, 30): 427}
42


In [10]:
def combined_requests_from_and_to_depot_distance(locations, requests, depot_loc_ID, request_id1, request_id2):
    """
    Calculate the combined distances from the depot to the first request, between the two requests,
    and from the second request back to the depot.

    Args:
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - depot_loc_ID: The location ID of the depot.
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.

    Returns:
    - combined_distances: Dictionary storing the combined distances.
    """
    combined_distances = {}
    
    # Calculate distance from depot to first request
    depot_to_first_request_distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id1)
    
    # Calculate distance between the two consecutive requests
    consecutive_requests_distance = distance_between_consecutive_requests(locations, requests, request_id1, request_id2)
    
    # Calculate distance from second request back to depot
    second_request_to_depot_distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id2)
    
    # Store the combined distances in the dictionary
    combined_distances[(request_id1, request_id2)] = {
        'depot_to_first_request_distance': depot_to_first_request_distance,
        'consecutive_requests_distance': consecutive_requests_distance,
        'second_request_to_depot_distance': second_request_to_depot_distance
    }
    
    return combined_distances

# Example usage:
combined_distances = combined_requests_from_and_to_depot_distance(locations, requests, 1, 3, 4)
print(combined_distances)

def calculate_all_combined_distances(locations, requests, depot_loc_ID):
    """
    Calculate the combined distances from the depot to the first request, between the two requests,
    and from the second request back to the depot for all pairs of consecutive requests.

    Args:
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - depot_loc_ID: The location ID of the depot.

    Returns:
    - combined_distances: Dictionary storing the combined distances for all pairs of consecutive requests.
    """
    combined_distances = {}
    
    # Iterate through all pairs of consecutive requests
    for i in range(len(requests) - 1):
        request_id1 = requests[i][0]
        request_id2 = requests[i + 1][0]
        
        # Calculate the combined distances for the current pair of consecutive requests
        combined_distance = combined_requests_from_and_to_depot_distance(locations, requests, depot_loc_ID, request_id1, request_id2)
        
        # Store the combined distances in the dictionary
        combined_distances[(request_id1, request_id2)] = combined_distance
    
    return combined_distances

# Example usage:
combined_distances = calculate_all_combined_distances(locations, requests, 1)
print(combined_distances)

# Create a dictionary to store total distances for each pair
total_distances_for_pairs = {}

# Iterate through the dictionary and calculate the total distance for each pair
for pair, distances in combined_distances.items():
    total_distance_for_pair = 0
    for sub_pair, sub_distances in distances.items():
        total_distance_for_pair += sub_distances['depot_to_first_request_distance']
        total_distance_for_pair += sub_distances['consecutive_requests_distance']
        total_distance_for_pair += sub_distances['second_request_to_depot_distance']
    total_distances_for_pairs[pair] = total_distance_for_pair

# Print or access the dictionary as needed
print(total_distances_for_pairs)



{(3, 4): {'depot_to_first_request_distance': 169, 'consecutive_requests_distance': 244, 'second_request_to_depot_distance': 149}}
{(1, 2): {(1, 2): {'depot_to_first_request_distance': 361, 'consecutive_requests_distance': 42, 'second_request_to_depot_distance': 354}}, (2, 3): {(2, 3): {'depot_to_first_request_distance': 354, 'consecutive_requests_distance': 279, 'second_request_to_depot_distance': 169}}, (3, 4): {(3, 4): {'depot_to_first_request_distance': 169, 'consecutive_requests_distance': 244, 'second_request_to_depot_distance': 149}}, (4, 5): {(4, 5): {'depot_to_first_request_distance': 149, 'consecutive_requests_distance': 184, 'second_request_to_depot_distance': 119}}, (5, 6): {(5, 6): {'depot_to_first_request_distance': 119, 'consecutive_requests_distance': 0, 'second_request_to_depot_distance': 119}}, (6, 7): {(6, 7): {'depot_to_first_request_distance': 119, 'consecutive_requests_distance': 61, 'second_request_to_depot_distance': 169}}, (7, 8): {(7, 8): {'depot_to_first_reque

In [11]:
def check_feasibility_for_all_pairs(total_distances_for_pairs, TRUCK_MAX_DISTANCE):
    """
    Check the feasibility of total distances for all pairs.

    Args:
    - total_distances_for_pairs: Dictionary where keys are pairs and values are total distances.
    - MAX_TRUCK_CAPACITY: Maximum truck capacity.

    Returns:
    - feasibility_dict: Dictionary where keys are pairs and values are boolean indicating feasibility.
    """
    feasibility_dict = {}
    for pair, total_distance in total_distances_for_pairs.items():
        feasibility_dict[pair] = total_distance <= TRUCK_MAX_DISTANCE
    return feasibility_dict

# Check feasibility for all pairs
feasibility_dict = check_feasibility_for_all_pairs(total_distances_for_pairs, TRUCK_MAX_DISTANCE)

# Print feasibility for each pair
for pair, feasible in feasibility_dict.items():
    print(f"Pair {pair}: Feasible? {feasible}")


Pair (1, 2): Feasible? False
Pair (2, 3): Feasible? False
Pair (3, 4): Feasible? True
Pair (4, 5): Feasible? True
Pair (5, 6): Feasible? True
Pair (6, 7): Feasible? True
Pair (7, 8): Feasible? True
Pair (8, 9): Feasible? True
Pair (9, 10): Feasible? True
Pair (10, 11): Feasible? True
Pair (11, 12): Feasible? True
Pair (12, 13): Feasible? True
Pair (13, 14): Feasible? True
Pair (14, 15): Feasible? True
Pair (15, 16): Feasible? True
Pair (16, 17): Feasible? False
Pair (17, 18): Feasible? False
Pair (18, 19): Feasible? True
Pair (19, 20): Feasible? False
Pair (20, 21): Feasible? True
Pair (21, 22): Feasible? False
Pair (22, 23): Feasible? True
Pair (23, 24): Feasible? True
Pair (24, 25): Feasible? True
Pair (25, 26): Feasible? True
Pair (26, 27): Feasible? False
Pair (27, 28): Feasible? True
Pair (28, 29): Feasible? True
Pair (29, 30): Feasible? False


In [12]:
def get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append([request_id])
            current_truck_weight = request_weight
            current_delivery_day = start_day
        elif current_truck_weight + request_weight <= TRUCK_CAPACITY and start_day == current_delivery_day:
            # Check if adding the current request exceeds the truck capacity and if delivery day matches
            combined_request_ids = current_truck_route[-1] + [request_id]
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= TRUCK_CAPACITY:
                current_truck_route[-1] = combined_request_ids
                current_truck_weight = combined_weight
            else:
                truck_routes.append((current_truck_route, current_delivery_day))
                current_truck_route = [[request_id]]
                current_truck_weight = request_weight
                current_delivery_day = start_day
        else:
            # If adding the current request exceeds the truck capacity or delivery day does not match,
            # start a new truck route with the current request
            truck_routes.append((current_truck_route, current_delivery_day))
            current_truck_route = [[request_id]]
            current_truck_weight = request_weight
            current_delivery_day = start_day
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes


def calculate_delivery_days(truck_routes):
    delivery_days = {}
    for truck_route, delivery_day in truck_routes:
        for request_ids in truck_route:
            for request_id in request_ids:
                delivery_days[request_id] = delivery_day
    return delivery_days



# Calculate truck routes
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)

# Calculate delivery days using truck routes
delivery_days = calculate_delivery_days(truck_routes)


print(truck_routes)



[([[1]], 1), ([[2]], 1), ([[3, 4]], 1), ([[5, 6]], 2), ([[7]], 2), ([[8]], 2), ([[9]], 2), ([[10]], 2), ([[11, 12]], 3), ([[13, 14]], 3), ([[15]], 3), ([[16, 17]], 4), ([[18]], 4), ([[19]], 4), ([[20]], 4), ([[21]], 4), ([[22]], 5), ([[23]], 5), ([[24]], 5), ([[25]], 6), ([[26, 27, 28, 29]], 6), ([[30]], 6)]


In [13]:
def get_truck_routes_with_constraints(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    current_truck_distance = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        # Calculate the distance from the depot to the current location
        depot_to_location_distance = calculate_request_distance(requests, locations, depot_loc_ID, location_id)
        
        # Check if the distance calculation was successful
        if depot_to_location_distance is None:
            print(f"Error: Unable to calculate distance between depot and request {request_id}")
            continue
        
        # Check if adding the current request meets all constraints
        if (current_truck_weight + request_weight <= TRUCK_CAPACITY and 
            current_truck_distance + depot_to_location_distance <= TRUCK_MAX_DISTANCE and
            start_day == current_delivery_day):
            # If constraints are satisfied, add the request to the current truck route
            if current_truck_route:
                combined_request_ids = current_truck_route[-1] + [request_id]
                if all(feasibility_dict[(req_id1, req_id2)] for req_id1 in current_truck_route[-1] for req_id2 in [request_id]):
                    current_truck_route[-1] = combined_request_ids
                    current_truck_weight += request_weight
                    current_truck_distance += depot_to_location_distance
                else:
                    truck_routes.append((current_truck_route, current_delivery_day))
                    current_truck_route = [[request_id]]  # Start a new truck route
                    current_truck_weight = request_weight
                    current_truck_distance = depot_to_location_distance
                    current_delivery_day = start_day
            else:
                current_truck_route.append([request_id])  # Add the request ID
                current_truck_weight += request_weight
                current_truck_distance += depot_to_location_distance
        else:
            # If constraints are not satisfied, start a new truck route with the current request
            if current_truck_route:
                truck_routes.append((current_truck_route, current_delivery_day))  # Append current truck route
            current_truck_route = [[request_id]]  # Start a new truck route
            current_truck_weight = request_weight
            current_truck_distance = depot_to_location_distance
            current_delivery_day = start_day
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes

# Calculate truck routes with constraints
truck_routes_with_constraints = get_truck_routes_with_constraints(
    locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict
)
print(truck_routes_with_constraints)



Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 8: ID=2, Coordinates=(742, 453)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 3: ID=11, Coordinates=(551, 677)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 11: ID=4, Coordinates=(595, 438)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 4: ID=4, Coordinates=(595, 438)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 12: ID=12, Coordinates=(571, 620)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 12: ID=12, Coordinates=(571, 620)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 11: ID=4, Coordinates=(595, 438)
Location corresponding to R

In [14]:
def constrained_truck_routes_with_depot(truck_routes_with_constraints, depot_loc_ID):
    truck_routes_with_depot = []
    for truck_route, delivery_day in truck_routes_with_constraints:
        # Add depot at the beginning and end of the truck route
        truck_route_with_depot = [depot_loc_ID] + truck_route + [depot_loc_ID]
        truck_routes_with_depot.append((truck_route_with_depot, delivery_day))
    return truck_routes_with_depot

# Calculate truck routes with constraints
truck_routes_with_constraints = get_truck_routes_with_constraints(
    locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict
)

# Add depot to truck routes
truck_routes_with_depot = constrained_truck_routes_with_depot(truck_routes_with_constraints, depot_loc_ID)

print(truck_routes_with_depot)


Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 8: ID=2, Coordinates=(742, 453)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 3: ID=11, Coordinates=(551, 677)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 11: ID=4, Coordinates=(595, 438)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 4: ID=4, Coordinates=(595, 438)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 12: ID=12, Coordinates=(571, 620)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 12: ID=12, Coordinates=(571, 620)
Location corresponding to Request ID 1: ID=8, Coordinates=(326, 473)
Location corresponding to Request ID 11: ID=4, Coordinates=(595, 438)
Location corresponding to R

In [15]:
def extract_truck_routes(truck_routes_with_depot):
    extracted_routes = []
    for route, delivery_day in truck_routes_with_depot:
        request_ids = []
        for segment in route:
            if isinstance(segment, list):  # If segment is a list, extract request IDs from it
                request_ids.extend(segment)
        extracted_routes.append((request_ids, delivery_day))
    return extracted_routes


# Example usage
truck_routes_extracted = extract_truck_routes(truck_routes_with_depot)
print()
print(truck_routes_extracted)

def convert_truck_routes_to_locations(truck_routes_with_depot, requests):
    truck_routes_locations = []
    for route, delivery_day in truck_routes_with_depot:
        location_route = [1]  # Start at location ID 1
        for segment in route:
            if isinstance(segment, list):  # Check if segment is a list (request IDs)
                for request_id in segment:
                    location_id = requests[request_id - 1][1]  # Location ID of the request
                    location_route.append(location_id)
        location_route.append(1)  # End at location ID 1
        truck_routes_locations.append((location_route, delivery_day))
    return truck_routes_locations


# Convert truck routes with depot to locations
truck_routes_locations = convert_truck_routes_to_locations(truck_routes_with_depot, requests)
print()
print(truck_routes_locations)

def calculate_truck_distances(locations, truck_routes_locations):
    truck_distances_per_day = {}
    total_truck_distance = 0
    
    for idx, (route, _) in enumerate(truck_routes_locations, start=1):
        total_distance = 0
        for i in range(len(route) - 1):
            loc1 = locations[route[i] - 1]
            loc2 = locations[route[i + 1] - 1]
            distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
            total_distance += distance
        total_distance = round(total_distance, 2)  # Round to 2 decimal places
        truck_distances_per_day[idx] = total_distance
        total_truck_distance += total_distance
    
    total_truck_distance = round(total_truck_distance, 2)  # Round total distance
    return truck_distances_per_day, total_truck_distance

# Calculate truck distances
truck_distances_per_day, total_truck_distance = calculate_truck_distances(locations, truck_routes_locations)

# Print distances for each truck per day
print("Truck Distances Per Day:")
for truck_idx, distance in truck_distances_per_day.items():
    print(f"Truck {truck_idx}: {distance:.2f}")

# Print total truck distance
print(f"\nTotal Truck Distance: {total_truck_distance:.2f}")


## weet niet of deze nog klopt na aanpassing delivery/installation days
def calculate_idle_days(requests, delivery_days):
    """
    Calculate the idle days for each machine based on delivery and installation days.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, delivery_start_day, delivery_end_day, machine_kind_id, num_requested_machines]
    - installations: List of installations where each installation is represented as a list [installation_day, machine_kind_id]

    Returns:
    - idle_days: Dictionary where keys are machine kind IDs and values are lists of idle days.
    """
    idle_days = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
        machine_kind_id = requests[request_id - 1][4]  # Machine kind ID is at index 4
        idle_days[machine_kind_id] = max(0, installation_day - delivery_day)
    return idle_days

# Calculate idle days
idle_days = calculate_idle_days(requests, delivery_days)





# Function to calculate penalties for idle days
def calculate_penalties(delivery_days, requests):
    penalties = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        machine_kind_id = requests[request_id - 1][4]
        idle_days = max(0, installation_day - delivery_day - 1)  # Subtract 1 for the day of delivery
        if machine_kind_id not in penalties:
            penalties[machine_kind_id] = 0
        penalties[machine_kind_id] += idle_days
    return penalties


# Function to implement constraints regarding maximum consecutive working days and total distance per day
def apply_constraints(technician_assignments):
    # Implement constraints here
    pass




[([1], 1), ([2], 1), ([3, 4], 1), ([5, 6], 2), ([7], 2), ([8], 2), ([9], 2), ([10], 2), ([11, 12], 3), ([13], 3), ([14, 15], 3), ([16], 4), ([17], 4), ([18], 4), ([19], 4), ([20], 4), ([21], 4), ([22], 5), ([23], 5), ([24], 5), ([25], 6), ([26], 6), ([27, 28], 6), ([29], 6), ([30], 6)]

[([1, 8, 1], 1), ([1, 3, 1], 1), ([1, 11, 4, 1], 1), ([1, 12, 12, 1], 2), ([1, 11, 1], 2), ([1, 2, 1], 2), ([1, 2, 1], 2), ([1, 10, 1], 2), ([1, 4, 12, 1], 3), ([1, 15, 1], 3), ([1, 9, 14, 1], 3), ([1, 15, 1], 4), ([1, 3, 1], 4), ([1, 4, 1], 4), ([1, 2, 1], 4), ([1, 8, 1], 4), ([1, 6, 1], 4), ([1, 5, 1], 5), ([1, 4, 1], 5), ([1, 13, 1], 5), ([1, 5, 1], 6), ([1, 4, 1], 6), ([1, 3, 12, 1], 6), ([1, 14, 1], 6), ([1, 6, 1], 6)]
Truck Distances Per Day:
Truck 1: 720.84
Truck 2: 706.83
Truck 3: 559.86
Truck 4: 237.19
Truck 5: 336.88
Truck 6: 257.60
Truck 7: 257.60
Truck 8: 255.54
Truck 9: 450.58
Truck 10: 484.29
Truck 11: 526.71
Truck 12: 484.29
Truck 13: 706.83
Truck 14: 296.82
Truck 15: 257.60
Truck 16: 72

In [16]:
def check_truck_distances(truck_distances_per_day, TRUCK_MAX_DISTANCE):
    exceeded_trucks = []
    for truck_idx, distance in truck_distances_per_day.items():
        if distance > TRUCK_MAX_DISTANCE:
            exceeded_trucks.append(truck_idx)
    return exceeded_trucks

# Call this method after calculating truck distances
exceeded_trucks = check_truck_distances(truck_distances_per_day, TRUCK_MAX_DISTANCE)

if exceeded_trucks:
    print("The following trucks exceeded the maximum distance:")
    for truck_idx in exceeded_trucks:
        print(f"Truck {truck_idx}")
else:
    print("All trucks are within the maximum distance.")


All trucks are within the maximum distance.


In [17]:
def get_truck_routes_with_depot(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = [depot_loc_ID]  # Start at the depot
    current_truck_weight = 0
    current_truck_distance = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        # Calculate the distance from the last location in the route to the current location
        if len(current_truck_route) > 1:
            loc1 = locations[current_truck_route[-1] - 1]
            loc2 = locations[location_id - 1]
            distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
            current_truck_distance += distance
        
        if current_truck_weight + request_weight <= TRUCK_CAPACITY and start_day == current_delivery_day and current_truck_distance <= TRUCK_MAX_DISTANCE:
            # Check if adding the current request exceeds the truck capacity and if delivery day matches
            current_truck_route.append(location_id)  # Add the location ID to the route
            current_truck_weight += request_weight
        else:
            # End the current route and start a new one
            truck_routes.append((current_truck_route, current_delivery_day))
            current_truck_route = [depot_loc_ID, location_id]  # Start a new truck route
            current_truck_weight = request_weight
            current_delivery_day = start_day
            current_truck_distance = 0
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes



def calculate_truck_distance(locations, route):
    total_distance = 0
    for i in range(len(route) - 1):
        loc1 = locations[route[i] - 1]
        loc2 = locations[route[i + 1] - 1]
        distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
        total_distance += distance
    return round(total_distance, 2)

# Recalculate truck routes with depot considering maximum distance
truck_routes_with_depot = get_truck_routes_with_depot(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines)


In [18]:
def get_location_for_request(request):
    return requests[request-1][1]

def nearest_neighbor(point, unvisited_requests, locations):

    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_request in unvisited_requests:
        request_location_id = get_location_for_request(candidate_request)
        if request_location_id != point:
            candidate_coordinates = get_location_coordinates(request_location_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = request_location_id
    return nearest_point


def get_requests_for_location(requests, location_id):
    """
    Get the list of request IDs corresponding to the given location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - location_id: The location ID for which the request IDs are to be retrieved.

    Returns:
    - request_ids: A list of request IDs corresponding to the given location ID.
    """
    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids
for location in locations:
    location_id = location[0]
    location_request  = [get_requests_for_location(requests, location_id)]
    print(location_request)
    

technician_details = {}

# Define function to print technician details
def print_technician_details(technicians):
    global technician_details
    technician_details.clear()  # Clear existing data
    for technician in technicians:
        tech_id = technician[0]
        max_distance = technician[2]
        max_installations = technician[3]
        technician_details[tech_id] = {'max_distance': max_distance, 'max_installations': max_installations}


# Call function to print technician details
print_technician_details(technicians)

# Now you can print technician_details
print(technician_details)

# Initialize max_distance and max_installations dictionaries
max_distance = {tech_id: details['max_distance'] for tech_id, details in technician_details.items()}
max_installations = {tech_id: details['max_installations'] for tech_id, details in technician_details.items()}



def initialize_home_locations(technicians):
    global home_location_technician
    home_location_technician = {tech[0]: tech[1] for tech in technicians}
    
    # Print the home locations
    print("Home Locations for Technicians:")
    for tech_id, home_location in home_location_technician.items():
        print(f"Technician {tech_id}: Location {home_location}")
    
    return home_location_technician
initialize_home_locations(technicians)
print(home_location_technician)


def nearest_neighbor_for_technician_home(technicians, unvisited_requests, locations):
    home_location_technician = initialize_home_locations(technicians)
    
    for technician_id, home_location in home_location_technician.items():
        nearest_location = nearest_neighbor(home_location, unvisited_requests, locations)
        print(f"The closest location for technician {technician_id} is location {nearest_location}.")
        
        
def transform_skill_set(technicians, technician_id):
    for technician in technicians:
        if technician[0] == technician_id:
            skill_set = technician[4:]  # Get the skill set of the technician (from index 4 onwards)
            transformed_skills = []
            
            # Iterate through the skill_set with index
            for index, value in enumerate(skill_set):
                # If the value is non-zero, append the 1-based index
                if value != 0:
                    transformed_skills.append(index + 1)
                else:
                    # If the value is zero, append zero
                    transformed_skills.append(0)
            
            return transformed_skills
    return None 

def doable_request_skillbased(technicians, requests):
    doable_requests = {}  # Dictionary to store which technician can handle which requests

    for technician in technicians:
        technician_id = technician[0]
        skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
        
        for request in requests:
            request_id = request[0]
            machine_type = request[4]  # Fifth entry denotes the machine type

            # Check if the technician has this machine type in their skill set
            if machine_type in skill_set:
                if technician_id not in doable_requests:
                    doable_requests[technician_id] = []
                doable_requests[technician_id].append(request_id)
    
    return doable_requests


doable_requests = doable_request_skillbased(technicians, requests)
doable_requests_copy = doable_request_skillbased(technicians, requests)
for technician_id, request_ids in doable_requests.items():
    print(f"Technician {technician_id} can handle requests: {request_ids}")
transformed_skills = transform_skill_set(technicians, technician_id)
if transformed_skills is not None:
    print(f"Technician {technician_id}: {transformed_skills}")
else:
    print(f"Technician {technician_id} not found.")

    
def check_location_requests_doable(technicians, requests, locations):
    # Dictionary to store the result for each location
    location_results = {}
    
    for location in locations:
        location_id = location[0]
        location_requests = [request for request in requests if request[1] == location_id]
        location_doable_requests = {}
        
        for technician in technicians:
            technician_id = technician[0]
            skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
            
            doable_requests = []
            for request in location_requests:
                machine_type = request[4]  # Fifth entry denotes the machine type
                
                # Check if the technician has this machine type in their skill set
                if machine_type in skill_set:
                    doable_requests.append(request[0])
            
            if doable_requests:
                location_doable_requests[technician_id] = doable_requests
        
        location_results[location_id] = location_doable_requests
    
    return location_results

location_results = check_location_requests_doable(technicians, requests, locations)
for location_id, doable_requests in location_results.items():
    print(f"Location {location_id}:")
    for technician_id, request_ids in doable_requests.items():
        print(f"- Technician {technician_id} can handle requests: {request_ids}")

unvisited_requests = [1, 2, 3, 4,5]
current_location = 2

nearest = nearest_neighbor(current_location, unvisited_requests, locations)
print(f"The closest location for {current_location} is location {nearest}.")

# Call the new method
nearest_neighbor_for_technician_home(technicians, unvisited_requests, locations)




[[]]
[[8, 9, 19]]
[[2, 17, 27]]
[[4, 11, 18, 23, 26]]
[[22, 25]]
[[21, 30]]
[[]]
[[1, 20]]
[[14]]
[[10]]
[[3, 7]]
[[5, 6, 12, 28]]
[[24]]
[[15, 29]]
[[13, 16]]
[[]]
{1: {'max_distance': 212, 'max_installations': 5}, 2: {'max_distance': 503, 'max_installations': 7}, 3: {'max_distance': 808, 'max_installations': 6}, 4: {'max_distance': 506, 'max_installations': 7}, 5: {'max_distance': 108, 'max_installations': 8}, 6: {'max_distance': 135, 'max_installations': 8}, 7: {'max_distance': 119, 'max_installations': 9}, 8: {'max_distance': 507, 'max_installations': 8}, 9: {'max_distance': 414, 'max_installations': 7}, 10: {'max_distance': 208, 'max_installations': 7}}
Home Locations for Technicians:
Technician 1: Location 7
Technician 2: Location 1
Technician 3: Location 13
Technician 4: Location 6
Technician 5: Location 1
Technician 6: Location 12
Technician 7: Location 7
Technician 8: Location 8
Technician 9: Location 16
Technician 10: Location 11
{1: 7, 2: 1, 3: 13, 4: 6, 5: 1, 6: 12, 7: 7, 8

In [19]:
print(doable_requests_copy)

{1: [3, 5, 6, 10, 14, 17, 23, 24, 26, 27, 28, 29], 2: [3, 5, 6, 10, 14, 17, 23, 24, 26, 27, 28, 29], 3: [1, 2, 4, 7, 8, 9, 11, 13, 15, 18, 19, 20, 21, 22, 25], 4: [1, 2, 4, 7, 8, 9, 11, 12, 13, 15, 16, 18, 19, 20, 21, 22, 25, 30], 5: [1, 2, 4, 7, 8, 9, 11, 13, 15, 18, 19, 20, 21, 22, 25], 6: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 7: [3, 5, 6, 10, 14, 17, 23, 24, 26, 27, 28, 29], 8: [3, 5, 6, 10, 12, 14, 16, 17, 23, 24, 26, 27, 28, 29, 30], 9: [12, 16, 30], 10: [1, 2, 4, 7, 8, 9, 11, 13, 15, 18, 19, 20, 21, 22, 25]}


In [20]:
def rare_requests(doable_requests_copy):
    """
    Identify the requests that can be handled by the fewest number of technicians.

    Args:
    - doable_requests: Dictionary where keys are technician IDs and values are lists of request IDs they can handle.

    Returns:
    - rare_requests_dict: Dictionary where keys are request IDs and values are the number of technicians that can handle them.
    """
    # Create a dictionary to count how many technicians can handle each request
    request_count = {}

    # Iterate through the doable_requests dictionary
    for tech_id, requests in doable_requests_copy.items():
        for request_id in requests:
            if request_id in request_count:
                request_count[request_id] += 1
            else:
                request_count[request_id] = 1

    # Debug print to check request counts
    print("Request counts:", request_count)

    # If request_count is empty, return an empty dictionary
    if not request_count:
        return {}

    # Find the minimum count
    min_count = min(request_count.values())

    # Debug print to check the minimum count
    print("Minimum count:", min_count)

    # Identify the requests that have this minimum count
    rare_requests_dict = {req_id: count for req_id, count in request_count.items() if count == min_count}

    return rare_requests_dict


rare_requests_dict = rare_requests(doable_requests_copy)

print("Rare Requests:")
for request_id, count in rare_requests_dict.items():
    print(f"Request {request_id} can be handled by {count} technician(s)")


Request counts: {3: 5, 5: 5, 6: 5, 10: 5, 14: 5, 17: 5, 23: 5, 24: 5, 26: 5, 27: 5, 28: 5, 29: 5, 1: 5, 2: 5, 4: 5, 7: 5, 8: 5, 9: 5, 11: 5, 13: 5, 15: 5, 18: 5, 19: 5, 20: 5, 21: 5, 22: 5, 25: 5, 12: 4, 16: 4, 30: 4}
Minimum count: 4
Rare Requests:
Request 12 can be handled by 4 technician(s)
Request 16 can be handled by 4 technician(s)
Request 30 can be handled by 4 technician(s)


In [21]:
def extract_machine_count(requests):
    machine_count = {}
    for req in requests:
        machine_count[req[0]] = req[5]
    return machine_count


machine_count = extract_machine_count(requests)

# Print machine count for each request
print("Machine Count for Each Request:")
for request_id, count in machine_count.items():
    print(f"Request {request_id}: {count} machines")



Machine Count for Each Request:
Request 1: 1 machines
Request 2: 2 machines
Request 3: 2 machines
Request 4: 1 machines
Request 5: 1 machines
Request 6: 2 machines
Request 7: 1 machines
Request 8: 2 machines
Request 9: 2 machines
Request 10: 2 machines
Request 11: 1 machines
Request 12: 1 machines
Request 13: 1 machines
Request 14: 2 machines
Request 15: 1 machines
Request 16: 2 machines
Request 17: 1 machines
Request 18: 1 machines
Request 19: 2 machines
Request 20: 2 machines
Request 21: 1 machines
Request 22: 2 machines
Request 23: 3 machines
Request 24: 3 machines
Request 25: 2 machines
Request 26: 1 machines
Request 27: 2 machines
Request 28: 1 machines
Request 29: 1 machines
Request 30: 1 machines


In [22]:
def extract_delivery_schedule(truck_routes):
    delivery_schedule = {}
    for route in truck_routes:
        requests, delivery_day = route
        for req in requests:
            for request_id in req:
                delivery_schedule[request_id] = delivery_day
    return delivery_schedule

def print_delivery_schedule(delivery_schedule):
    print("Delivery Schedule:")
    for request_id, delivery_day in sorted(delivery_schedule.items()):
        print(f"Request {request_id}: Delivery Day {delivery_day}")


delivery_schedule = extract_delivery_schedule(truck_routes)

# Print delivery schedule for all requests
print_delivery_schedule(delivery_schedule)

# Determine the number of requests
num_requests = len(delivery_schedule)
print("Number of Requests:", num_requests)

# example to acces delivery day for 1 specific request
print("Delivery Day of Request 1:", delivery_schedule.get(1))  # Example for request 1

def extract_assignment_days(truck_routes):
    assignment_schedule = {}
    for route in truck_routes:
        requests, delivery_day = route
        for req in requests:
            for request_id in req:
                assignment_schedule[request_id] = delivery_day + 1
    return assignment_schedule

# Example usage:
assignment_schedule = extract_assignment_days(truck_routes)

# Print assignment days for all requests
print("Assignment Days:")
for request_id, assignment_day in sorted(assignment_schedule.items()):
    print(f"Request {request_id}: Assignment Day {assignment_day}")

# Example to access assignment day for a specific request
print("Assignment Day of Request 1:", assignment_schedule.get(1))  # Example for request 1





Delivery Schedule:
Request 1: Delivery Day 1
Request 2: Delivery Day 1
Request 3: Delivery Day 1
Request 4: Delivery Day 1
Request 5: Delivery Day 2
Request 6: Delivery Day 2
Request 7: Delivery Day 2
Request 8: Delivery Day 2
Request 9: Delivery Day 2
Request 10: Delivery Day 2
Request 11: Delivery Day 3
Request 12: Delivery Day 3
Request 13: Delivery Day 3
Request 14: Delivery Day 3
Request 15: Delivery Day 3
Request 16: Delivery Day 4
Request 17: Delivery Day 4
Request 18: Delivery Day 4
Request 19: Delivery Day 4
Request 20: Delivery Day 4
Request 21: Delivery Day 4
Request 22: Delivery Day 5
Request 23: Delivery Day 5
Request 24: Delivery Day 5
Request 25: Delivery Day 6
Request 26: Delivery Day 6
Request 27: Delivery Day 6
Request 28: Delivery Day 6
Request 29: Delivery Day 6
Request 30: Delivery Day 6
Number of Requests: 30
Delivery Day of Request 1: 1
Assignment Days:
Request 1: Assignment Day 2
Request 2: Assignment Day 2
Request 3: Assignment Day 2
Request 4: Assignment Day 2

In [23]:
print(assignment_schedule)

{1: 2, 2: 2, 3: 2, 4: 2, 5: 3, 6: 3, 7: 3, 8: 3, 9: 3, 10: 3, 11: 4, 12: 4, 13: 4, 14: 4, 15: 4, 16: 5, 17: 5, 18: 5, 19: 5, 20: 5, 21: 5, 22: 6, 23: 6, 24: 6, 25: 7, 26: 7, 27: 7, 28: 7, 29: 7, 30: 7}


In [24]:
def doable_request_skillbased(technicians, requests):
    doable_requests = {}  # Dictionary to store which technician can handle which requests

    for technician in technicians:
        technician_id = technician[0]
        skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
        
        for request in requests:
            request_id = request[0]
            machine_type = request[4]  # Fifth entry denotes the machine type

            # Check if the technician has this machine type in their skill set
            if machine_type in skill_set:
                if technician_id not in doable_requests:
                    doable_requests[technician_id] = []
                doable_requests[technician_id].append(request_id)
    
    return doable_requests


doable_requests = doable_request_skillbased(technicians, requests)

print(doable_requests[1])

for technician_id, request_ids in doable_requests.items():
    print(f"Technician {technician_id} can handle requests: {request_ids}")
    transformed_skills = transform_skill_set(technicians, technician_id)
    if transformed_skills is not None:
        print(f"Technician {technician_id}: skill set {transformed_skills}")
    else:
        print(f"Technician {technician_id} not found.")

    


[3, 5, 6, 10, 14, 17, 23, 24, 26, 27, 28, 29]
Technician 1 can handle requests: [3, 5, 6, 10, 14, 17, 23, 24, 26, 27, 28, 29]
Technician 1: skill set [0, 2, 0]
Technician 2 can handle requests: [3, 5, 6, 10, 14, 17, 23, 24, 26, 27, 28, 29]
Technician 2: skill set [0, 2, 0]
Technician 3 can handle requests: [1, 2, 4, 7, 8, 9, 11, 13, 15, 18, 19, 20, 21, 22, 25]
Technician 3: skill set [0, 0, 3]
Technician 4 can handle requests: [1, 2, 4, 7, 8, 9, 11, 12, 13, 15, 16, 18, 19, 20, 21, 22, 25, 30]
Technician 4: skill set [1, 0, 3]
Technician 5 can handle requests: [1, 2, 4, 7, 8, 9, 11, 13, 15, 18, 19, 20, 21, 22, 25]
Technician 5: skill set [0, 0, 3]
Technician 6 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Technician 6: skill set [1, 2, 3]
Technician 7 can handle requests: [3, 5, 6, 10, 14, 17, 23, 24, 26, 27, 28, 29]
Technician 7: skill set [0, 2, 0]
Technician 8 can handle requests: [3, 5, 6, 10, 12,

In [25]:
def home_location_technician_coordinates(home_location_technician, locations):
    """
    Get the coordinates of technicians' home locations.

    Args:
    - home_location_technician: Dictionary where keys are technician IDs and values are their home location IDs.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - home_coordinates: Dictionary where keys are technician IDs and values are tuples (x, y) representing their home location coordinates.
    """
    home_coordinates = {}
    for tech_id, location_id in home_location_technician.items():
        coordinates = get_location_coordinates(location_id, locations)
        if coordinates:
            home_coordinates[tech_id] = coordinates
    return home_coordinates

home_coordinates = home_location_technician_coordinates(home_location_technician, locations)
print("Technicians' home coordinates:")
for tech_id, coordinates in home_coordinates.items():
    print(f"Technician {tech_id}: {coordinates}")

Technicians' home coordinates:
Technician 1: (608, 353)
Technician 2: (675, 563)
Technician 3: (411, 373)
Technician 4: (316, 466)
Technician 5: (675, 563)
Technician 6: (571, 620)
Technician 7: (608, 353)
Technician 8: (326, 473)
Technician 9: (237, 349)
Technician 10: (551, 677)


In [26]:

def request_location_coordinates(requests, locations):
    """
    Get the coordinates of each request location.

    Args:
    - requests: List of requests where each request is represented as [request_id, _, location_id, _, _, _].
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - request_coordinates: Dictionary where keys are request IDs and values are tuples (x, y) representing their location coordinates.
    """
    request_coordinates = {}
    for request in requests:
        request_id, location_id, *_ = request
        coordinates = get_location_coordinates(location_id, locations)
        if coordinates:
            request_coordinates[request_id] = coordinates
    return request_coordinates

request_coordinates = request_location_coordinates(requests, locations)
print("Requests' location coordinates:")
for request_id, coordinates in request_coordinates.items():
    print(f"Request {request_id}: {coordinates}")

    
def get_request_location_ids(requests):
    """
    Extract and print the location IDs of each request.

    Args:
    - requests: List of lists where each inner list represents a request.

    Returns:
    - location_ids: A list of location IDs extracted from the requests.
    """
    location_ids = [request[1] for request in requests]
    return location_ids


# Get all request location IDs
location_ids = get_request_location_ids(requests)

# Print all request location IDs
for i, location_id in enumerate(location_ids, start=1):
    print(f"Request ID {i}: Location ID {location_id}")


Requests' location coordinates:
Request 1: (326, 473)
Request 2: (325, 514)
Request 3: (551, 677)
Request 4: (595, 438)
Request 5: (571, 620)
Request 6: (571, 620)
Request 7: (551, 677)
Request 8: (742, 453)
Request 9: (742, 453)
Request 10: (689, 690)
Request 11: (595, 438)
Request 12: (571, 620)
Request 13: (543, 360)
Request 14: (533, 437)
Request 15: (742, 455)
Request 16: (543, 360)
Request 17: (325, 514)
Request 18: (595, 438)
Request 19: (742, 453)
Request 20: (326, 473)
Request 21: (316, 466)
Request 22: (583, 353)
Request 23: (595, 438)
Request 24: (411, 373)
Request 25: (583, 353)
Request 26: (595, 438)
Request 27: (325, 514)
Request 28: (571, 620)
Request 29: (742, 455)
Request 30: (316, 466)
Request ID 1: Location ID 8
Request ID 2: Location ID 3
Request ID 3: Location ID 11
Request ID 4: Location ID 4
Request ID 5: Location ID 12
Request ID 6: Location ID 12
Request ID 7: Location ID 11
Request ID 8: Location ID 2
Request ID 9: Location ID 2
Request ID 10: Location ID 10
R

In [27]:
def distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id):
    """
    Calculate the distance between a home location and a request location.

    Args:
    - home_coordinates: Dictionary containing home IDs as keys and their coordinates as values.
    - request_coordinates: Dictionary containing request IDs as keys and their coordinates as values.
    - home_id: ID of the home.
    - request_id: ID of the request.

    Returns:
    - distance: The Euclidean distance between the home and request locations.
    """
    home_location = home_coordinates.get(home_id)
    request_location = request_coordinates.get(request_id)
    if home_location is None:
        raise ValueError(f"Home ID {home_id} not found in home_coordinates.")
    if request_location is None:
        raise ValueError(f"Request ID {request_id} not found in request_coordinates.")
    
    return calculate_distance(home_location, request_location)

def print_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates):
    """
    Print distances between each technician's home and each request.

    Args:
    - home_coordinates: Dictionary containing home IDs as keys and their coordinates as values.
    - request_coordinates: Dictionary containing request IDs as keys and their coordinates as values.
    """
    for home_id, home_location in home_coordinates.items():
        for request_id, request_location in request_coordinates.items():
            distance = distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id)
            print(f"Distance between home {home_id} and request {request_id}: {distance}")

print_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates)

Distance between home 1 and request 1: 307
Distance between home 1 and request 2: 326
Distance between home 1 and request 3: 329
Distance between home 1 and request 4: 86
Distance between home 1 and request 5: 270
Distance between home 1 and request 6: 270
Distance between home 1 and request 7: 329
Distance between home 1 and request 8: 168
Distance between home 1 and request 9: 168
Distance between home 1 and request 10: 347
Distance between home 1 and request 11: 86
Distance between home 1 and request 12: 270
Distance between home 1 and request 13: 66
Distance between home 1 and request 14: 113
Distance between home 1 and request 15: 169
Distance between home 1 and request 16: 66
Distance between home 1 and request 17: 326
Distance between home 1 and request 18: 86
Distance between home 1 and request 19: 168
Distance between home 1 and request 20: 307
Distance between home 1 and request 21: 314
Distance between home 1 and request 22: 25
Distance between home 1 and request 23: 86
Dist

In [28]:
def calculate_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates):
    """
    Calculate and store distances between each technician's home and each request.

    Args:
    - home_coordinates: Dictionary containing home IDs as keys and their coordinates as values.
    - request_coordinates: Dictionary containing request IDs as keys and their coordinates as values.

    Returns:
    - distances: Dictionary with keys as (home_id, request_id) tuples and values as distances.
    """
    distances = {}
    for home_id, home_location in home_coordinates.items():
        for request_id, request_location in request_coordinates.items():
            distance = distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id)
            distances[(home_id, request_id)] = distance
            print(f"Distance between home {home_id} and request {request_id}: {distance}")
    return distances


distances = calculate_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates)



Distance between home 1 and request 1: 307
Distance between home 1 and request 2: 326
Distance between home 1 and request 3: 329
Distance between home 1 and request 4: 86
Distance between home 1 and request 5: 270
Distance between home 1 and request 6: 270
Distance between home 1 and request 7: 329
Distance between home 1 and request 8: 168
Distance between home 1 and request 9: 168
Distance between home 1 and request 10: 347
Distance between home 1 and request 11: 86
Distance between home 1 and request 12: 270
Distance between home 1 and request 13: 66
Distance between home 1 and request 14: 113
Distance between home 1 and request 15: 169
Distance between home 1 and request 16: 66
Distance between home 1 and request 17: 326
Distance between home 1 and request 18: 86
Distance between home 1 and request 19: 168
Distance between home 1 and request 20: 307
Distance between home 1 and request 21: 314
Distance between home 1 and request 22: 25
Distance between home 1 and request 23: 86
Dist

In [29]:
print(home_location_technician)
print(home_coordinates)
print(home_coordinates.get(1))
print(request_coordinates)
distances[(1,9)]


{1: 7, 2: 1, 3: 13, 4: 6, 5: 1, 6: 12, 7: 7, 8: 8, 9: 16, 10: 11}
{1: (608, 353), 2: (675, 563), 3: (411, 373), 4: (316, 466), 5: (675, 563), 6: (571, 620), 7: (608, 353), 8: (326, 473), 9: (237, 349), 10: (551, 677)}
(608, 353)
{1: (326, 473), 2: (325, 514), 3: (551, 677), 4: (595, 438), 5: (571, 620), 6: (571, 620), 7: (551, 677), 8: (742, 453), 9: (742, 453), 10: (689, 690), 11: (595, 438), 12: (571, 620), 13: (543, 360), 14: (533, 437), 15: (742, 455), 16: (543, 360), 17: (325, 514), 18: (595, 438), 19: (742, 453), 20: (326, 473), 21: (316, 466), 22: (583, 353), 23: (595, 438), 24: (411, 373), 25: (583, 353), 26: (595, 438), 27: (325, 514), 28: (571, 620), 29: (742, 455), 30: (316, 466)}


168

In [30]:
def check_max_distance_exceeded(distances, max_distance):
    """
    Check if the round trip distance for each technician exceeds their maximum distance.

    Args:
    - distances: Dictionary containing distances between each technician's home and each request.
    - max_distance: Dictionary containing maximum distances for each technician.

    Returns:
    - results: Dictionary containing tuples of (home_id, request_id) as keys and boolean values indicating
               whether the round trip distance exceeds the maximum distance for the corresponding technician.
    """
    results = {}
    for (home_id, request_id), distance in distances.items():
        round_trip_distance = 2 * distance
        max_dist = max_distance.get(home_id, float('inf'))
        exceeds_max_distance = round_trip_distance > max_dist
        results[(home_id, request_id)] = exceeds_max_distance
    return results

# Assuming distances is already calculated from the previous code
exceeded_distances = check_max_distance_exceeded(distances, max_distance)

# Print the results
for (home_id, request_id), exceeds in exceeded_distances.items():
    status = "exceeds" if exceeds else "does not exceed"
    print(f"Round trip distance for home {home_id} to request {request_id} {'exceeds' if exceeds else 'does not exceed'} the max distance.")

    
## ONDERSTAANDE GEBRUIKT OM TE CHECKEN OF HET ECHT EXCEEDE OF NIET MET PRINTEN VAN DISTANCE AND MAX DISTANCE, MAG WEG
## def check_max_distance_exceeded(distances, max_distance):
##    """
##    Check if the round trip distance for each technician exceeds their maximum distance.

##      Args:
##      - distances: Dictionary containing distances between each technician's home and each request.
##      - max_distance: Dictionary containing maximum distances for each technician.

##      Returns:
##      - results: Dictionary containing tuples of (home_id, request_id) as keys and boolean values indicating
##                 whether the round trip distance exceeds the maximum distance for the corresponding technician.
##      """
##      results = {}
##      for (home_id, request_id), distance in distances.items():
##          round_trip_distance = 2 * distance
##          max_dist = max_distance.get(home_id, float('inf'))
##          exceeds_max_distance = round_trip_distance > max_dist
##          results[(home_id, request_id)] = exceeds_max_distance
##      return results

##  # Assuming distances is already calculated from the previous code
##  exceeded_distances = check_max_distance_exceeded(distances, max_distance)

##  # Print the results
##  for (home_id, request_id), exceeds in exceeded_distances.items():
##      status = "exceeds" if exceeds else "does not exceed"
##      distance = distances[(home_id, request_id)]
##      max_dist = max_distance.get(home_id, float('inf'))
##      round_trip_distance = distance*2
##      exceeds_max_distance = round_trip_distance > max_dist
##      print(f"Round trip distance for home {home_id} to request {request_id} (distance is {round_trip_distance}) {'exceeds' if exceeds_max_distance else 'does not exceed'} the max distance (max distance of {max_dist}).")


Round trip distance for home 1 to request 1 exceeds the max distance.
Round trip distance for home 1 to request 2 exceeds the max distance.
Round trip distance for home 1 to request 3 exceeds the max distance.
Round trip distance for home 1 to request 4 does not exceed the max distance.
Round trip distance for home 1 to request 5 exceeds the max distance.
Round trip distance for home 1 to request 6 exceeds the max distance.
Round trip distance for home 1 to request 7 exceeds the max distance.
Round trip distance for home 1 to request 8 exceeds the max distance.
Round trip distance for home 1 to request 9 exceeds the max distance.
Round trip distance for home 1 to request 10 exceeds the max distance.
Round trip distance for home 1 to request 11 does not exceed the max distance.
Round trip distance for home 1 to request 12 exceeds the max distance.
Round trip distance for home 1 to request 13 does not exceed the max distance.
Round trip distance for home 1 to request 14 exceeds the max d

In [31]:
exceeded_distances

{(1, 1): True,
 (1, 2): True,
 (1, 3): True,
 (1, 4): False,
 (1, 5): True,
 (1, 6): True,
 (1, 7): True,
 (1, 8): True,
 (1, 9): True,
 (1, 10): True,
 (1, 11): False,
 (1, 12): True,
 (1, 13): False,
 (1, 14): True,
 (1, 15): True,
 (1, 16): False,
 (1, 17): True,
 (1, 18): False,
 (1, 19): True,
 (1, 20): True,
 (1, 21): True,
 (1, 22): False,
 (1, 23): False,
 (1, 24): True,
 (1, 25): False,
 (1, 26): False,
 (1, 27): True,
 (1, 28): True,
 (1, 29): True,
 (1, 30): True,
 (2, 1): True,
 (2, 2): True,
 (2, 3): False,
 (2, 4): False,
 (2, 5): False,
 (2, 6): False,
 (2, 7): False,
 (2, 8): False,
 (2, 9): False,
 (2, 10): False,
 (2, 11): False,
 (2, 12): False,
 (2, 13): False,
 (2, 14): False,
 (2, 15): False,
 (2, 16): False,
 (2, 17): True,
 (2, 18): False,
 (2, 19): False,
 (2, 20): True,
 (2, 21): True,
 (2, 22): False,
 (2, 23): False,
 (2, 24): True,
 (2, 25): False,
 (2, 26): False,
 (2, 27): True,
 (2, 28): False,
 (2, 29): False,
 (2, 30): True,
 (3, 1): False,
 (3, 2): Fa

In [32]:
def doable_requests_distance_based(exceeded_distances):
    """
    Check which requests each technician can handle based on the distances and maximum distances.

    Args:
    - exceeded_distances: Dictionary containing tuples of (home_id, request_id) as keys and boolean values indicating
                          whether the round trip distance exceeds the maximum distance for the corresponding technician.

    Returns:
    - doable_requests: Dictionary where keys are technician ids and values are lists of requests that the technician can handle.
    """
    doable_requests = {}
    for (home_id, request_id), exceeds in exceeded_distances.items():
        if not exceeds:
            if home_id not in doable_requests:
                doable_requests[home_id] = []
            doable_requests[home_id].append(request_id)
    return doable_requests

# Get doable requests based on distance
doable_requests_distance_based = doable_requests_distance_based(exceeded_distances)

# Print the result
for technician, requests in doable_requests_distance_based.items():
    print(f"Technician {technician} can handle requests: {requests}")


Technician 1 can handle requests: [4, 11, 13, 16, 18, 22, 23, 25, 26]
Technician 2 can handle requests: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 22, 23, 25, 26, 28, 29]
Technician 3 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Technician 4 can handle requests: [1, 2, 13, 14, 16, 17, 20, 21, 24, 27, 30]
Technician 6 can handle requests: [3, 5, 6, 7, 12, 28]
Technician 7 can handle requests: [22, 25]
Technician 8 can handle requests: [1, 2, 13, 14, 16, 17, 20, 21, 24, 27, 30]
Technician 9 can handle requests: [1, 2, 17, 20, 21, 24, 27, 30]
Technician 10 can handle requests: [3, 5, 6, 7, 12, 28]


In [33]:
# dit returned een list met requests die de technicain kan doen based op zn skills en op zn distance vanaf home


def doable_requests_double_check(doable_requests_distance_based, doable_requests):
    double_check_requests = {}

    # Iterate through technician IDs
    for tech_id in doable_requests_distance_based.keys():
        # Find common requests between both dictionaries
        common_requests = set(doable_requests_distance_based[tech_id]) & set(doable_requests[tech_id])
        double_check_requests[tech_id] = list(common_requests)

    return double_check_requests



double_check_requests = doable_requests_double_check(doable_requests_distance_based, doable_requests)

print(double_check_requests)


{1: [26, 23], 2: [3, 5, 6, 10, 14, 23, 26, 28, 29], 3: [1, 2, 4, 7, 8, 9, 11, 13, 15, 18, 19, 20, 21, 22, 25], 4: [1, 2, 13, 16, 20, 21, 30], 6: [3, 5, 6, 7, 12, 28], 7: [], 8: [14, 16, 17, 24, 27, 30], 9: [30], 10: [7]}


In [34]:
def rare_requests_double_checked(double_check_requests):
    # Count how many technicians can handle each request and keep track of the technicians
    request_count = {}
    request_technicians = {}
    for tech_id, tech_requests in double_check_requests.items():
        for request in tech_requests:
            request_count[request] = request_count.get(request, 0) + 1
            request_technicians.setdefault(request, []).append(tech_id)
    
    # Find the minimum count
    min_count = min(request_count.values()) if request_count else 0
    
    # Identify the requests that have this minimum count
    rare_requests_dict = {req_id: {'count': count, 'technicians': request_technicians[req_id]} for req_id, count in request_count.items() if count == min_count}
    
    return rare_requests_dict

# Example usage:
rare_requests_dict = rare_requests_double_checked(double_check_requests)
print("Rare Requests:")
for request_id, data in rare_requests_dict.items():
    count = data['count']
    technicians_copy = data['technicians']
    print(f"Request {request_id} can be handled by the least number of technicians ({count} technicians): Technicians {technicians_copy}")
print(technicians)
print(rare_requests_dict)

Rare Requests:
Request 10 can be handled by the least number of technicians (1 technicians): Technicians [2]
Request 29 can be handled by the least number of technicians (1 technicians): Technicians [2]
Request 4 can be handled by the least number of technicians (1 technicians): Technicians [3]
Request 8 can be handled by the least number of technicians (1 technicians): Technicians [3]
Request 9 can be handled by the least number of technicians (1 technicians): Technicians [3]
Request 11 can be handled by the least number of technicians (1 technicians): Technicians [3]
Request 15 can be handled by the least number of technicians (1 technicians): Technicians [3]
Request 18 can be handled by the least number of technicians (1 technicians): Technicians [3]
Request 19 can be handled by the least number of technicians (1 technicians): Technicians [3]
Request 22 can be handled by the least number of technicians (1 technicians): Technicians [3]
Request 25 can be handled by the least number of

In [35]:
def technician_rare_request_count(double_check_requests, rare_requests):
    # Initialize a dictionary to store the rare request count for each technician
    technician_rare_counts = {tech_id: 0 for tech_id in double_check_requests.keys()}
    
    # Iterate through rare requests and update the count for each technician
    for req_id, data in rare_requests.items():
        technicians = data['technicians']
        for tech_id in technicians:
            technician_rare_counts[tech_id] += 1
    
    return technician_rare_counts

# Example usage:
technician_rare_counts = technician_rare_request_count(double_check_requests, rare_requests_dict)
print("Technician Rare Request Counts:")
for tech_id, count in technician_rare_counts.items():
    print(f"Technician {tech_id} has {count} rare requests.")

print(technician_rare_counts)
print(rare_requests)
print(rare_requests_dict)

Technician Rare Request Counts:
Technician 1 has 0 rare requests.
Technician 2 has 2 rare requests.
Technician 3 has 9 rare requests.
Technician 4 has 0 rare requests.
Technician 6 has 1 rare requests.
Technician 7 has 0 rare requests.
Technician 8 has 3 rare requests.
Technician 9 has 0 rare requests.
Technician 10 has 0 rare requests.
{1: 0, 2: 2, 3: 9, 4: 0, 6: 1, 7: 0, 8: 3, 9: 0, 10: 0}
<function rare_requests at 0x7fe5d6346290>
{10: {'count': 1, 'technicians': [2]}, 29: {'count': 1, 'technicians': [2]}, 4: {'count': 1, 'technicians': [3]}, 8: {'count': 1, 'technicians': [3]}, 9: {'count': 1, 'technicians': [3]}, 11: {'count': 1, 'technicians': [3]}, 15: {'count': 1, 'technicians': [3]}, 18: {'count': 1, 'technicians': [3]}, 19: {'count': 1, 'technicians': [3]}, 22: {'count': 1, 'technicians': [3]}, 25: {'count': 1, 'technicians': [3]}, 12: {'count': 1, 'technicians': [6]}, 17: {'count': 1, 'technicians': [8]}, 24: {'count': 1, 'technicians': [8]}, 27: {'count': 1, 'technicians': 

In [36]:
def find_requests_with_same_day_and_loc(assignment_schedule, request_coordinates, location_ids):
    """
    Find requests with the same location ID and assignment day.

    Args:
    - assignment_schedule: Dictionary where keys are request IDs and values are assignment days.
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.
    - location_ids: List of location IDs extracted from the requests.

    Returns:
    - requests_same_day_loc: Dictionary where keys are tuples (location ID, assignment day) and values are lists of request IDs.
    """
    requests_same_day_loc = {}
    for request_id, assignment_day in assignment_schedule.items():
        location_id = location_ids[request_id - 1]  # Adjusting index since request_id starts from 1
        if location_id is not None:
            key = (location_id, assignment_day)
            requests_same_day_loc.setdefault(key, []).append(request_id)
    return {key: requests for key, requests in requests_same_day_loc.items() if len(requests) > 1}

# Example usage:
requests_same_day_loc = find_requests_with_same_day_and_loc(assignment_schedule, request_coordinates, location_ids)

# Print requests with the same location and assignment day
print("Requests with the Same Location ID and Assignment Day:")
for (location_id, assignment_day), requests in requests_same_day_loc.items():
    print(f"Location ID {location_id}, Assignment Day {assignment_day}: Requests {requests}")


Requests with the Same Location ID and Assignment Day:
Location ID 12, Assignment Day 3: Requests [5, 6]
Location ID 2, Assignment Day 3: Requests [8, 9]


In [37]:
requests_same_day_loc

{(12, 3): [5, 6], (2, 3): [8, 9]}

In [38]:
def find_requests_with_same_day(assignment_schedule, request_coordinates):
    """
    Find requests with the same assignment day and print their location.

    Args:
    - assignment_schedule: Dictionary where keys are request IDs and values are assignment days.
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.

    Returns:
    - requests_same_day: Dictionary where keys are assignment days and values are lists of tuples (request ID, location coordinates).
    """
    requests_same_day = {}
    for request_id, assignment_day in assignment_schedule.items():
        location = request_coordinates.get(request_id)
        if location is not None:
            requests_same_day.setdefault(assignment_day, []).append((request_id, location))
    return {day: requests for day, requests in requests_same_day.items() if len(requests) > 1}

# Example usage:
requests_same_day = find_requests_with_same_day(assignment_schedule, request_coordinates)

# Print requests with the same assignment day and their locations
print("Requests with the Same Assignment Day and Their Locations:")
for assignment_day, requests in requests_same_day.items():
    print(f"Assignment Day {assignment_day}:")
    for request_id, location in requests:
        print(f"Request {request_id}: Location {location}")

print(requests_same_day)

Requests with the Same Assignment Day and Their Locations:
Assignment Day 2:
Request 1: Location (326, 473)
Request 2: Location (325, 514)
Request 3: Location (551, 677)
Request 4: Location (595, 438)
Assignment Day 3:
Request 5: Location (571, 620)
Request 6: Location (571, 620)
Request 7: Location (551, 677)
Request 8: Location (742, 453)
Request 9: Location (742, 453)
Request 10: Location (689, 690)
Assignment Day 4:
Request 11: Location (595, 438)
Request 12: Location (571, 620)
Request 13: Location (543, 360)
Request 14: Location (533, 437)
Request 15: Location (742, 455)
Assignment Day 5:
Request 16: Location (543, 360)
Request 17: Location (325, 514)
Request 18: Location (595, 438)
Request 19: Location (742, 453)
Request 20: Location (326, 473)
Request 21: Location (316, 466)
Assignment Day 6:
Request 22: Location (583, 353)
Request 23: Location (595, 438)
Request 24: Location (411, 373)
Assignment Day 7:
Request 25: Location (583, 353)
Request 26: Location (595, 438)
Request 27

In [39]:


def find_all_request_pairs_same_day(requests_same_day):
    """
    Find all possible pairs of requests with the same assignment day.

    Args:
    - requests_same_day: Dictionary where keys are assignment days and values are lists of tuples (request ID, location coordinates).

    Returns:
    - all_request_pairs: Dictionary where keys are assignment days and values are lists of tuples (request ID 1, request ID 2).
    """
    all_request_pairs = {}
    for day, requests in requests_same_day.items():
        pairs = combinations([request[0] for request in requests], 2)
        all_request_pairs[day] = list(pairs)
    return all_request_pairs

# Example usage:
all_request_pairs = find_all_request_pairs_same_day(requests_same_day)

# Print all possible pairs of requests for each day
print("All Possible Pairs of Requests for Each Assignment Day:")
for assignment_day, pairs in all_request_pairs.items():
    print(f"Assignment Day {assignment_day}:")
    for pair in pairs:
        print(f"Request Pair {pair}")

all_request_pairs


All Possible Pairs of Requests for Each Assignment Day:
Assignment Day 2:
Request Pair (1, 2)
Request Pair (1, 3)
Request Pair (1, 4)
Request Pair (2, 3)
Request Pair (2, 4)
Request Pair (3, 4)
Assignment Day 3:
Request Pair (5, 6)
Request Pair (5, 7)
Request Pair (5, 8)
Request Pair (5, 9)
Request Pair (5, 10)
Request Pair (6, 7)
Request Pair (6, 8)
Request Pair (6, 9)
Request Pair (6, 10)
Request Pair (7, 8)
Request Pair (7, 9)
Request Pair (7, 10)
Request Pair (8, 9)
Request Pair (8, 10)
Request Pair (9, 10)
Assignment Day 4:
Request Pair (11, 12)
Request Pair (11, 13)
Request Pair (11, 14)
Request Pair (11, 15)
Request Pair (12, 13)
Request Pair (12, 14)
Request Pair (12, 15)
Request Pair (13, 14)
Request Pair (13, 15)
Request Pair (14, 15)
Assignment Day 5:
Request Pair (16, 17)
Request Pair (16, 18)
Request Pair (16, 19)
Request Pair (16, 20)
Request Pair (16, 21)
Request Pair (17, 18)
Request Pair (17, 19)
Request Pair (17, 20)
Request Pair (17, 21)
Request Pair (18, 19)
Request

{2: [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)],
 3: [(5, 6),
  (5, 7),
  (5, 8),
  (5, 9),
  (5, 10),
  (6, 7),
  (6, 8),
  (6, 9),
  (6, 10),
  (7, 8),
  (7, 9),
  (7, 10),
  (8, 9),
  (8, 10),
  (9, 10)],
 4: [(11, 12),
  (11, 13),
  (11, 14),
  (11, 15),
  (12, 13),
  (12, 14),
  (12, 15),
  (13, 14),
  (13, 15),
  (14, 15)],
 5: [(16, 17),
  (16, 18),
  (16, 19),
  (16, 20),
  (16, 21),
  (17, 18),
  (17, 19),
  (17, 20),
  (17, 21),
  (18, 19),
  (18, 20),
  (18, 21),
  (19, 20),
  (19, 21),
  (20, 21)],
 6: [(22, 23), (22, 24), (23, 24)],
 7: [(25, 26),
  (25, 27),
  (25, 28),
  (25, 29),
  (25, 30),
  (26, 27),
  (26, 28),
  (26, 29),
  (26, 30),
  (27, 28),
  (27, 29),
  (27, 30),
  (28, 29),
  (28, 30),
  (29, 30)]}

In [40]:
def find_request_pairs_same_day(request_coordinates, requests_same_day):
    """
    Find pairs of requests with the same assignment day and calculate the distance between them.

    Args:
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.
    - requests_same_day: Dictionary where keys are assignment days and values are lists of tuples (request ID, location coordinates).

    Returns:
    - request_pairs_distance: Dictionary where keys are assignment days and values are lists of tuples ((request ID 1, request ID 2), distance).
    """
    request_pairs_distance = {}
    for day, requests in requests_same_day.items():
        pairs = combinations(requests, 2)
        pairs_with_distance = []
        for pair in pairs:
            request_id1, coord1 = pair[0]
            request_id2, coord2 = pair[1]
            distance = calculate_distance(coord1, coord2)
            pairs_with_distance.append(((request_id1, request_id2), distance))
        request_pairs_distance[day] = pairs_with_distance
    return request_pairs_distance

# Example usage:
request_pairs_distance = find_request_pairs_same_day(request_coordinates, requests_same_day)

# Print request pairs with the same assignment day and their distances
print("Request Pairs with the Same Assignment Day and Their Distances:")
for day, pairs_distance in request_pairs_distance.items():
    print(f"Assignment Day {day}:")
    for pair, distance in pairs_distance:
        request_id1, request_id2 = pair
        print(f"Request Pair ({request_id1}, {request_id2}): Distance {distance}")


Request Pairs with the Same Assignment Day and Their Distances:
Assignment Day 2:
Request Pair (1, 2): Distance 42
Request Pair (1, 3): Distance 304
Request Pair (1, 4): Distance 272
Request Pair (2, 3): Distance 279
Request Pair (2, 4): Distance 281
Request Pair (3, 4): Distance 244
Assignment Day 3:
Request Pair (5, 6): Distance 0
Request Pair (5, 7): Distance 61
Request Pair (5, 8): Distance 240
Request Pair (5, 9): Distance 240
Request Pair (5, 10): Distance 138
Request Pair (6, 7): Distance 61
Request Pair (6, 8): Distance 240
Request Pair (6, 9): Distance 240
Request Pair (6, 10): Distance 138
Request Pair (7, 8): Distance 295
Request Pair (7, 9): Distance 295
Request Pair (7, 10): Distance 139
Request Pair (8, 9): Distance 0
Request Pair (8, 10): Distance 243
Request Pair (9, 10): Distance 243
Assignment Day 4:
Request Pair (11, 12): Distance 184
Request Pair (11, 13): Distance 94
Request Pair (11, 14): Distance 63
Request Pair (11, 15): Distance 148
Request Pair (12, 13): Dista

In [41]:
def calculate_round_trip_distances(home_coordinates, request_pairs_distance, request_coordinates):
    """
    Calculate the round-trip distance from the technician's home depot to the first request,
    to the second request, and back to the home depot.

    Args:
    - home_coordinates: Dictionary where keys are technician IDs and values are tuples representing the coordinates of the home depots.
    - request_pairs_distance: Dictionary where keys are assignment days and values are lists of tuples ((request ID 1, request ID 2), distance).
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.

    Returns:
    - technician_round_trip_distances: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance).
    """
    technician_round_trip_distances = {}
    for technician, home_coord in home_coordinates.items():
        round_trip_distances = {}
        for day, pairs_distance in request_pairs_distance.items():
            day_distances = []
            for pair, distance in pairs_distance:
                request_id1, request_id2 = pair
                coord1 = request_coordinates[request_id1]
                coord2 = request_coordinates[request_id2]
                distance_to_first = calculate_distance(home_coord, coord1)
                distance_from_second = calculate_distance(coord2, home_coord)
                round_trip_distance = distance_to_first + distance + distance_from_second
                day_distances.append((pair, round_trip_distance))
            round_trip_distances[day] = day_distances
        technician_round_trip_distances[technician] = round_trip_distances
    return technician_round_trip_distances


request_pairs_distance = find_request_pairs_same_day(request_coordinates, requests_same_day)
technician_round_trip_distances = calculate_round_trip_distances(home_coordinates, request_pairs_distance, request_coordinates)

# Print the round-trip distances for each technician
for technician, round_trip_distances in technician_round_trip_distances.items():
    print(f"Technician {technician}:")
    for day, pairs_distance in round_trip_distances.items():
        print(f"  Assignment Day {day}:")
        for pair, distance in pairs_distance:
            request_id1, request_id2 = pair
            print(f"    Request Pair ({request_id1}, {request_id2}): Round-Trip Distance {distance}")

technician_round_trip_distances

Technician 1:
  Assignment Day 2:
    Request Pair (1, 2): Round-Trip Distance 675
    Request Pair (1, 3): Round-Trip Distance 940
    Request Pair (1, 4): Round-Trip Distance 665
    Request Pair (2, 3): Round-Trip Distance 934
    Request Pair (2, 4): Round-Trip Distance 693
    Request Pair (3, 4): Round-Trip Distance 659
  Assignment Day 3:
    Request Pair (5, 6): Round-Trip Distance 540
    Request Pair (5, 7): Round-Trip Distance 660
    Request Pair (5, 8): Round-Trip Distance 678
    Request Pair (5, 9): Round-Trip Distance 678
    Request Pair (5, 10): Round-Trip Distance 755
    Request Pair (6, 7): Round-Trip Distance 660
    Request Pair (6, 8): Round-Trip Distance 678
    Request Pair (6, 9): Round-Trip Distance 678
    Request Pair (6, 10): Round-Trip Distance 755
    Request Pair (7, 8): Round-Trip Distance 792
    Request Pair (7, 9): Round-Trip Distance 792
    Request Pair (7, 10): Round-Trip Distance 815
    Request Pair (8, 9): Round-Trip Distance 336
    Request 

{1: {2: [((1, 2), 675),
   ((1, 3), 940),
   ((1, 4), 665),
   ((2, 3), 934),
   ((2, 4), 693),
   ((3, 4), 659)],
  3: [((5, 6), 540),
   ((5, 7), 660),
   ((5, 8), 678),
   ((5, 9), 678),
   ((5, 10), 755),
   ((6, 7), 660),
   ((6, 8), 678),
   ((6, 9), 678),
   ((6, 10), 755),
   ((7, 8), 792),
   ((7, 9), 792),
   ((7, 10), 815),
   ((8, 9), 336),
   ((8, 10), 758),
   ((9, 10), 758)],
  4: [((11, 12), 540),
   ((11, 13), 246),
   ((11, 14), 262),
   ((11, 15), 403),
   ((12, 13), 598),
   ((12, 14), 570),
   ((12, 15), 677),
   ((13, 14), 257),
   ((13, 15), 456),
   ((14, 15), 492)],
  5: [((16, 17), 659),
   ((16, 18), 246),
   ((16, 19), 454),
   ((16, 20), 618),
   ((16, 21), 631),
   ((17, 18), 693),
   ((17, 19), 916),
   ((17, 20), 675),
   ((17, 21), 689),
   ((18, 19), 402),
   ((18, 20), 665),
   ((18, 21), 681),
   ((19, 20), 892),
   ((19, 21), 909),
   ((20, 21), 634)],
  6: [((22, 23), 197), ((22, 24), 398), ((23, 24), 481)],
  7: [((25, 26), 197),
   ((25, 27), 656

In [42]:
def find_lowest_distance_pairs(technician_round_trip_distances):
    """
    Find the two request pairs with the lowest round-trip distances for each technician on each day.

    Args:
    - technician_round_trip_distances: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance).

    Returns:
    - lowest_distance_pairs: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of the two tuples ((request ID 1, request ID 2), round-trip distance) with the lowest distances.
    """
    lowest_distance_pairs = {}
    for technician, round_trip_distances in technician_round_trip_distances.items():
        tech_lowest_pairs = {}
        for day, pairs_distance in round_trip_distances.items():
            # Sort the pairs by distance and take the two with the lowest distance
            sorted_pairs = sorted(pairs_distance, key=lambda x: x[1])[:2]
            tech_lowest_pairs[day] = sorted_pairs
        lowest_distance_pairs[technician] = tech_lowest_pairs
    return lowest_distance_pairs

# Calculate the round-trip distances
request_pairs_distance = find_request_pairs_same_day(request_coordinates, requests_same_day)
technician_round_trip_distances = calculate_round_trip_distances(home_coordinates, request_pairs_distance, request_coordinates)

# Find the two pairs with the lowest distances for each technician on each day
lowest_distance_pairs = find_lowest_distance_pairs(technician_round_trip_distances)

# Print the lowest distance pairs for each technician
for technician, lowest_pairs in lowest_distance_pairs.items():
    print(f"Technician {technician}:")
    for day, pairs_distance in lowest_pairs.items():
        print(f"  Assignment Day {day}:")
        for pair, distance in pairs_distance:
            request_id1, request_id2 = pair
            print(f"    Request Pair ({request_id1}, {request_id2}): Round-Trip Distance {distance}")

lowest_distance_pairs


Technician 1:
  Assignment Day 2:
    Request Pair (3, 4): Round-Trip Distance 659
    Request Pair (1, 4): Round-Trip Distance 665
  Assignment Day 3:
    Request Pair (8, 9): Round-Trip Distance 336
    Request Pair (5, 6): Round-Trip Distance 540
  Assignment Day 4:
    Request Pair (11, 13): Round-Trip Distance 246
    Request Pair (13, 14): Round-Trip Distance 257
  Assignment Day 5:
    Request Pair (16, 18): Round-Trip Distance 246
    Request Pair (18, 19): Round-Trip Distance 402
  Assignment Day 6:
    Request Pair (22, 23): Round-Trip Distance 197
    Request Pair (22, 24): Round-Trip Distance 398
  Assignment Day 7:
    Request Pair (25, 26): Round-Trip Distance 197
    Request Pair (25, 29): Round-Trip Distance 383
Technician 2:
  Assignment Day 2:
    Request Pair (3, 4): Round-Trip Distance 562
    Request Pair (1, 2): Round-Trip Distance 757
  Assignment Day 3:
    Request Pair (5, 6): Round-Trip Distance 238
    Request Pair (8, 9): Round-Trip Distance 258
  Assignment

{1: {2: [((3, 4), 659), ((1, 4), 665)],
  3: [((8, 9), 336), ((5, 6), 540)],
  4: [((11, 13), 246), ((13, 14), 257)],
  5: [((16, 18), 246), ((18, 19), 402)],
  6: [((22, 23), 197), ((22, 24), 398)],
  7: [((25, 26), 197), ((25, 29), 383)]},
 2: {2: [((3, 4), 562), ((1, 2), 757)],
  3: [((5, 6), 238), ((8, 9), 258)],
  4: [((11, 14), 402), ((11, 15), 425)],
  5: [((18, 19), 426), ((16, 18), 486)],
  6: [((22, 23), 465), ((23, 24), 671)],
  7: [((26, 29), 425), ((26, 28), 452)]},
 3: {2: [((1, 2), 340), ((1, 4), 600)],
  3: [((5, 6), 590), ((8, 9), 682)],
  4: [((13, 14), 349), ((11, 14), 397)],
  5: [((20, 21), 278), ((17, 20), 340)],
  6: [((22, 24), 348), ((23, 24), 392)],
  7: [((27, 30), 348), ((25, 26), 456)]},
 4: {2: [((1, 2), 104), ((1, 4), 566)],
  3: [((5, 6), 596), ((5, 7), 675)],
  4: [((13, 14), 548), ((11, 14), 563)],
  5: [((20, 21), 26), ((17, 21), 98)],
  6: [((22, 24), 597), ((23, 24), 610)],
  7: [((27, 30), 98), ((26, 30), 562)]},
 5: {2: [((3, 4), 562), ((1, 2), 75

In [43]:
def can_handle_request_combinations(technician_round_trip_distances, max_distance):
    """
    Determine if a technician can handle the request combinations within their maximum travel distance.

    Args:
    - technician_round_trip_distances: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance).
    - max_distance: Dictionary where keys are technician IDs and values are the maximum travel distance for that technician.

    Returns:
    - feasible_combinations: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance) that are within the max distance.
    """
    feasible_combinations = {}
    for technician, round_trip_distances in technician_round_trip_distances.items():
        tech_feasible_pairs = {}
        for day, pairs_distance in round_trip_distances.items():
            feasible_pairs = [
                (pair, distance) for pair, distance in pairs_distance if distance <= max_distance.get(technician, float('inf'))
            ]
            tech_feasible_pairs[day] = feasible_pairs
        feasible_combinations[technician] = tech_feasible_pairs
    return feasible_combinations

# Calculate the round-trip distances
request_pairs_distance = find_request_pairs_same_day(request_coordinates, requests_same_day)
technician_round_trip_distances = calculate_round_trip_distances(home_coordinates, request_pairs_distance, request_coordinates)

# Find the two pairs with the lowest distances for each technician on each day
lowest_distance_pairs = find_lowest_distance_pairs(technician_round_trip_distances)


# Determine if the request combinations can be handled within the max travel distance
feasible_combinations = can_handle_request_combinations(technician_round_trip_distances, max_distance)

# Print the feasible combinations for each technician
for technician, feasible_pairs in feasible_combinations.items():
    print(f"Technician {technician}:")
    for day, pairs_distance in feasible_pairs.items():
        print(f"  Assignment Day {day}:")
        for pair, distance in pairs_distance:
            request_id1, request_id2 = pair
            print(f"    Request Pair ({request_id1}, {request_id2}): Round-Trip Distance {distance}")

feasible_combinations


Technician 1:
  Assignment Day 2:
  Assignment Day 3:
  Assignment Day 4:
  Assignment Day 5:
  Assignment Day 6:
    Request Pair (22, 23): Round-Trip Distance 197
  Assignment Day 7:
    Request Pair (25, 26): Round-Trip Distance 197
Technician 2:
  Assignment Day 2:
  Assignment Day 3:
    Request Pair (5, 6): Round-Trip Distance 238
    Request Pair (5, 7): Round-Trip Distance 349
    Request Pair (5, 8): Round-Trip Distance 488
    Request Pair (5, 9): Round-Trip Distance 488
    Request Pair (5, 10): Round-Trip Distance 385
    Request Pair (6, 7): Round-Trip Distance 349
    Request Pair (6, 8): Round-Trip Distance 488
    Request Pair (6, 9): Round-Trip Distance 488
    Request Pair (6, 10): Round-Trip Distance 385
    Request Pair (7, 10): Round-Trip Distance 436
    Request Pair (8, 9): Round-Trip Distance 258
    Request Pair (8, 10): Round-Trip Distance 500
    Request Pair (9, 10): Round-Trip Distance 500
  Assignment Day 4:
    Request Pair (11, 12): Round-Trip Distance 4

{1: {2: [], 3: [], 4: [], 5: [], 6: [((22, 23), 197)], 7: [((25, 26), 197)]},
 2: {2: [],
  3: [((5, 6), 238),
   ((5, 7), 349),
   ((5, 8), 488),
   ((5, 9), 488),
   ((5, 10), 385),
   ((6, 7), 349),
   ((6, 8), 488),
   ((6, 9), 488),
   ((6, 10), 385),
   ((7, 10), 436),
   ((8, 9), 258),
   ((8, 10), 500),
   ((9, 10), 500)],
  4: [((11, 12), 452),
   ((11, 13), 486),
   ((11, 14), 402),
   ((11, 15), 425),
   ((12, 14), 496),
   ((12, 15), 485)],
  5: [((16, 18), 486), ((18, 19), 426)],
  6: [((22, 23), 465)],
  7: [((25, 26), 465), ((26, 28), 452), ((26, 29), 425), ((28, 29), 485)]},
 3: {2: [((1, 2), 340),
   ((1, 3), 771),
   ((1, 4), 600),
   ((2, 3), 780),
   ((2, 4), 643),
   ((3, 4), 775)],
  3: [((5, 6), 590), ((5, 7), 691), ((6, 7), 691), ((8, 9), 682)],
  4: [((11, 12), 675),
   ((11, 13), 423),
   ((11, 14), 397),
   ((11, 15), 686),
   ((12, 13), 690),
   ((12, 14), 620),
   ((13, 14), 349),
   ((13, 15), 696),
   ((14, 15), 690)],
  5: [((16, 17), 566),
   ((16, 18),

In [44]:
def filter_combinations_by_skill_set(feasible_combinations, doable_requests_copy):
    """
    Filter the feasible request combinations to include only those where both requests are in the technician's skill set.

    Args:
    - feasible_combinations: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance).
    - doable_requests_copy: Dictionary where keys are technician IDs and values are lists of request IDs that the technician can handle.

    Returns:
    - skill_filtered_combinations: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance) that are within the technician's skill set.
    """
    skill_filtered_combinations = {}
    for technician, day_pairs in feasible_combinations.items():
        tech_skill_filtered = {}
        for day, pairs_distance in day_pairs.items():
            filtered_pairs = [
                (pair, distance) for pair, distance in pairs_distance
                if pair[0] in doable_requests_copy.get(technician, []) and pair[1] in doable_requests_copy.get(technician, [])
            ]
            tech_skill_filtered[day] = filtered_pairs
        skill_filtered_combinations[technician] = tech_skill_filtered
    return skill_filtered_combinations

# Example usage:

# Define feasible combinations based on max distance
feasible_combinations = can_handle_request_combinations(technician_round_trip_distances, max_distance)


# Filter the feasible combinations by the skill set of each technician
skill_filtered_combinations = filter_combinations_by_skill_set(feasible_combinations, doable_requests_copy)

# Print the skill filtered combinations for each technician
for technician, filtered_pairs in skill_filtered_combinations.items():
    print(f"Technician {technician}:")
    for day, pairs_distance in filtered_pairs.items():
        print(f"  Assignment Day {day}:")
        for pair, distance in pairs_distance:
            request_id1, request_id2 = pair
            print(f"    Request Pair ({request_id1}, {request_id2}): Round-Trip Distance {distance}")

skill_filtered_combinations


Technician 1:
  Assignment Day 2:
  Assignment Day 3:
  Assignment Day 4:
  Assignment Day 5:
  Assignment Day 6:
  Assignment Day 7:
Technician 2:
  Assignment Day 2:
  Assignment Day 3:
    Request Pair (5, 6): Round-Trip Distance 238
    Request Pair (5, 10): Round-Trip Distance 385
    Request Pair (6, 10): Round-Trip Distance 385
  Assignment Day 4:
  Assignment Day 5:
  Assignment Day 6:
  Assignment Day 7:
    Request Pair (26, 28): Round-Trip Distance 452
    Request Pair (26, 29): Round-Trip Distance 425
    Request Pair (28, 29): Round-Trip Distance 485
Technician 3:
  Assignment Day 2:
    Request Pair (1, 2): Round-Trip Distance 340
    Request Pair (1, 4): Round-Trip Distance 600
    Request Pair (2, 4): Round-Trip Distance 643
  Assignment Day 3:
    Request Pair (8, 9): Round-Trip Distance 682
  Assignment Day 4:
    Request Pair (11, 13): Round-Trip Distance 423
    Request Pair (11, 15): Round-Trip Distance 686
    Request Pair (13, 15): Round-Trip Distance 696
  Assig

{1: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 2: {2: [],
  3: [((5, 6), 238), ((5, 10), 385), ((6, 10), 385)],
  4: [],
  5: [],
  6: [],
  7: [((26, 28), 452), ((26, 29), 425), ((28, 29), 485)]},
 3: {2: [((1, 2), 340), ((1, 4), 600), ((2, 4), 643)],
  3: [((8, 9), 682)],
  4: [((11, 13), 423), ((11, 15), 686), ((13, 15), 696)],
  5: [((18, 19), 685), ((18, 20), 600), ((18, 21), 610), ((20, 21), 278)],
  6: [],
  7: []},
 4: {2: [((1, 2), 104)],
  3: [],
  4: [],
  5: [((16, 21), 502), ((20, 21), 26)],
  6: [],
  7: []},
 5: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 6: {2: [],
  3: [((5, 6), 0), ((5, 7), 122), ((6, 7), 122)],
  4: [],
  5: [],
  6: [],
  7: []},
 7: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 8: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [((27, 30), 104)]},
 9: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 10: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []}}

In [45]:
def filter_combinations_by_max_installations(skill_filtered_combinations, machine_count, max_installations):
    """
    Filter the skill filtered combinations to include only those where the total machine count
    for the request pair does not exceed the technician's maximum installation capacity.

    Args:
    - skill_filtered_combinations: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance) that are within the technician's skill set.
    - machine_count: Dictionary where keys are request IDs and values are the number of machines for each request.
    - max_installations: Dictionary where keys are technician IDs and values are the maximum number of machines the technician can install.

    Returns:
    - final_filtered_combinations: Dictionary where keys are technician IDs and values are dictionaries with keys as assignment days
      and values are lists of tuples ((request ID 1, request ID 2), round-trip distance) that are within the technician's skill set
      and within the maximum installation capacity.
    """
    final_filtered_combinations = {}
    for technician, day_pairs in skill_filtered_combinations.items():
        tech_filtered = {}
        for day, pairs_distance in day_pairs.items():
            filtered_pairs = [
                (pair, distance) for pair, distance in pairs_distance
                if machine_count[pair[0]] + machine_count[pair[1]] <= max_installations[technician]
            ]
            tech_filtered[day] = filtered_pairs
        final_filtered_combinations[technician] = tech_filtered
    return final_filtered_combinations

# Example usage:

# Define the skill filtered combinations
skill_filtered_combinations = filter_combinations_by_skill_set(feasible_combinations, doable_requests_copy)


# Filter the skill filtered combinations by the maximum installations for each technician
final_filtered_combinations = filter_combinations_by_max_installations(skill_filtered_combinations, machine_count, max_installations)

# Print the final filtered combinations for each technician
for technician, filtered_pairs in final_filtered_combinations.items():
    print(f"Technician {technician}:")
    for day, pairs_distance in filtered_pairs.items():
        print(f"  Assignment Day {day}:")
        for pair, distance in pairs_distance:
            request_id1, request_id2 = pair
            print(f"    Request Pair ({request_id1}, {request_id2}): Round-Trip Distance {distance}")

final_filtered_combinations


Technician 1:
  Assignment Day 2:
  Assignment Day 3:
  Assignment Day 4:
  Assignment Day 5:
  Assignment Day 6:
  Assignment Day 7:
Technician 2:
  Assignment Day 2:
  Assignment Day 3:
    Request Pair (5, 6): Round-Trip Distance 238
    Request Pair (5, 10): Round-Trip Distance 385
    Request Pair (6, 10): Round-Trip Distance 385
  Assignment Day 4:
  Assignment Day 5:
  Assignment Day 6:
  Assignment Day 7:
    Request Pair (26, 28): Round-Trip Distance 452
    Request Pair (26, 29): Round-Trip Distance 425
    Request Pair (28, 29): Round-Trip Distance 485
Technician 3:
  Assignment Day 2:
    Request Pair (1, 2): Round-Trip Distance 340
    Request Pair (1, 4): Round-Trip Distance 600
    Request Pair (2, 4): Round-Trip Distance 643
  Assignment Day 3:
    Request Pair (8, 9): Round-Trip Distance 682
  Assignment Day 4:
    Request Pair (11, 13): Round-Trip Distance 423
    Request Pair (11, 15): Round-Trip Distance 686
    Request Pair (13, 15): Round-Trip Distance 696
  Assig

{1: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 2: {2: [],
  3: [((5, 6), 238), ((5, 10), 385), ((6, 10), 385)],
  4: [],
  5: [],
  6: [],
  7: [((26, 28), 452), ((26, 29), 425), ((28, 29), 485)]},
 3: {2: [((1, 2), 340), ((1, 4), 600), ((2, 4), 643)],
  3: [((8, 9), 682)],
  4: [((11, 13), 423), ((11, 15), 686), ((13, 15), 696)],
  5: [((18, 19), 685), ((18, 20), 600), ((18, 21), 610), ((20, 21), 278)],
  6: [],
  7: []},
 4: {2: [((1, 2), 104)],
  3: [],
  4: [],
  5: [((16, 21), 502), ((20, 21), 26)],
  6: [],
  7: []},
 5: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 6: {2: [],
  3: [((5, 6), 0), ((5, 7), 122), ((6, 7), 122)],
  4: [],
  5: [],
  6: [],
  7: []},
 7: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 8: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [((27, 30), 104)]},
 9: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 10: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []}}

In [46]:
def can_handle_pairs_for_technicians(double_check_requests, all_request_pairs):
    """
    Check if each technician can handle both requests in all request pairs.

    Args:
    - double_check_requests: Dictionary where keys are technician IDs and values are lists of request IDs the technician can handle.
    - all_request_pairs: Dictionary where keys are assignment days and values are lists of tuples (request ID 1, request ID 2).

    Returns:
    - technician_capabilities: Dictionary where keys are technician IDs and values are dictionaries.
      Each dictionary has keys as assignment days and values as lists of pairs that the technician can handle.
    """
    technician_capabilities = {}

    for technician_id, tech_requests in double_check_requests.items():
        technician_capabilities[technician_id] = {}
        for day, pairs in all_request_pairs.items():
            request_pairs_for_technician = []
            for pair in pairs:
                if all(request in tech_requests for request in pair):
                    request_pairs_for_technician.append(pair)
            technician_capabilities[technician_id][day] = request_pairs_for_technician

    return technician_capabilities

# Example usage:
technician_capabilities = can_handle_pairs_for_technicians(double_check_requests, all_request_pairs)

# Print which technicians can handle which request pairs on which days
for technician, capabilities in technician_capabilities.items():
    print(f"Technician {technician}:")
    for day, pairs in capabilities.items():
        print(f"  Assignment Day {day}:")
        for pair in pairs:
            print(f"    Request Pair {pair}")

technician_capabilities

Technician 1:
  Assignment Day 2:
  Assignment Day 3:
  Assignment Day 4:
  Assignment Day 5:
  Assignment Day 6:
  Assignment Day 7:
Technician 2:
  Assignment Day 2:
  Assignment Day 3:
    Request Pair (5, 6)
    Request Pair (5, 10)
    Request Pair (6, 10)
  Assignment Day 4:
  Assignment Day 5:
  Assignment Day 6:
  Assignment Day 7:
    Request Pair (26, 28)
    Request Pair (26, 29)
    Request Pair (28, 29)
Technician 3:
  Assignment Day 2:
    Request Pair (1, 2)
    Request Pair (1, 4)
    Request Pair (2, 4)
  Assignment Day 3:
    Request Pair (7, 8)
    Request Pair (7, 9)
    Request Pair (8, 9)
  Assignment Day 4:
    Request Pair (11, 13)
    Request Pair (11, 15)
    Request Pair (13, 15)
  Assignment Day 5:
    Request Pair (18, 19)
    Request Pair (18, 20)
    Request Pair (18, 21)
    Request Pair (19, 20)
    Request Pair (19, 21)
    Request Pair (20, 21)
  Assignment Day 6:
  Assignment Day 7:
Technician 4:
  Assignment Day 2:
    Request Pair (1, 2)
  Assignme

{1: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 2: {2: [],
  3: [(5, 6), (5, 10), (6, 10)],
  4: [],
  5: [],
  6: [],
  7: [(26, 28), (26, 29), (28, 29)]},
 3: {2: [(1, 2), (1, 4), (2, 4)],
  3: [(7, 8), (7, 9), (8, 9)],
  4: [(11, 13), (11, 15), (13, 15)],
  5: [(18, 19), (18, 20), (18, 21), (19, 20), (19, 21), (20, 21)],
  6: [],
  7: []},
 4: {2: [(1, 2)],
  3: [],
  4: [],
  5: [(16, 20), (16, 21), (20, 21)],
  6: [],
  7: []},
 6: {2: [], 3: [(5, 6), (5, 7), (6, 7)], 4: [], 5: [], 6: [], 7: []},
 7: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 8: {2: [], 3: [], 4: [], 5: [(16, 17)], 6: [], 7: [(27, 30)]},
 9: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []},
 10: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []}}

In [47]:
print(double_check_requests)

{1: [26, 23], 2: [3, 5, 6, 10, 14, 23, 26, 28, 29], 3: [1, 2, 4, 7, 8, 9, 11, 13, 15, 18, 19, 20, 21, 22, 25], 4: [1, 2, 13, 16, 20, 21, 30], 6: [3, 5, 6, 7, 12, 28], 7: [], 8: [14, 16, 17, 24, 27, 30], 9: [30], 10: [7]}


In [48]:
print(all_request_pairs)

{2: [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)], 3: [(5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 7), (6, 8), (6, 9), (6, 10), (7, 8), (7, 9), (7, 10), (8, 9), (8, 10), (9, 10)], 4: [(11, 12), (11, 13), (11, 14), (11, 15), (12, 13), (12, 14), (12, 15), (13, 14), (13, 15), (14, 15)], 5: [(16, 17), (16, 18), (16, 19), (16, 20), (16, 21), (17, 18), (17, 19), (17, 20), (17, 21), (18, 19), (18, 20), (18, 21), (19, 20), (19, 21), (20, 21)], 6: [(22, 23), (22, 24), (23, 24)], 7: [(25, 26), (25, 27), (25, 28), (25, 29), (25, 30), (26, 27), (26, 28), (26, 29), (26, 30), (27, 28), (27, 29), (27, 30), (28, 29), (28, 30), (29, 30)]}


In [49]:
# method om te kijken of pairs in find_requests_with_same_day_and_loc are also pairs in doable_requests_double_check
def check_tech_req_same_day(double_check_requests, request_coordinates, assignment_schedule, location_ids):
    """
    Check if technicians can do requests with the same location and assignment day.

    Args:
    - double_check_requests: Dictionary where keys are technician IDs and values are lists of doable request IDs.
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.
    - assignment_schedule: Dictionary where keys are request IDs and values are assignment days.
    - location_ids: List of location IDs extracted from the requests.

    Returns:
    - tech_req_same_day: Dictionary where keys are tuples (technician ID, assignment day) and values are lists of request IDs.
    """
    tech_req_same_day = {}

    # Iterate through technician IDs
    for tech_id, requests in double_check_requests.items():
        # Initialize a dictionary to store requests with the same location and assignment day for the technician
        req_same_day = {}

        # Iterate over the technician's double-check requests
        for req_id in requests:
            location_id = location_ids[req_id - 1]  # Adjusting index since req_id starts from 1
            day = assignment_schedule[req_id]

            # Create a key for the location and assignment day
            key = (location_id, day)

            # Add the request to the dictionary under the corresponding key
            req_same_day.setdefault(key, []).append(req_id)

        # Filter out requests with the same location and assignment day
        req_same_day = {key: req_ids for key, req_ids in req_same_day.items() if len(req_ids) > 1}

        # Add the filtered dictionary to the tech_req_same_day dictionary
        if req_same_day:
            tech_req_same_day[tech_id] = req_same_day

    return tech_req_same_day

# Example usage:
tech_req_same_day = check_tech_req_same_day(double_check_requests, request_coordinates, assignment_schedule, location_ids)

# Print technicians who can do requests with the same location and assignment day
print("Technicians and Requests with the Same Location ID and Assignment Day:")
for tech_id, req_same_day in tech_req_same_day.items():
    for (location_id, assignment_day), req_ids in req_same_day.items():
        print(f"Technician ID {tech_id}, Location ID {location_id}, Assignment Day {assignment_day}: Requests {req_ids}")


Technicians and Requests with the Same Location ID and Assignment Day:
Technician ID 2, Location ID 12, Assignment Day 3: Requests [5, 6]
Technician ID 3, Location ID 2, Assignment Day 3: Requests [8, 9]
Technician ID 6, Location ID 12, Assignment Day 3: Requests [5, 6]


In [50]:
def possible_tech_combis_on_day(tech_req_same_day):
    """
    Create pairs of requests for technicians who can handle requests with the same location ID and assignment day.

    Args:
    - tech_req_same_day: Dictionary where keys are technician IDs and values are dictionaries of requests with the same location and assignment day.

    Returns:
    - tech_combis: List of tuples where each tuple contains a pair of requests for a technician.
    """
    tech_combis = []

    # Iterate through technicians
    for tech_id, req_same_day in tech_req_same_day.items():
        # Iterate through requests with the same location and assignment day
        for req_ids in req_same_day.values():
            # Create pairs of consecutive requests
            for i in range(len(req_ids) - 1):
                tech_combis.append((tech_id, req_ids[i], req_ids[i + 1]))

    return tech_combis

# Example usage:
tech_combis = possible_tech_combis_on_day(tech_req_same_day)

# Print possible technician pairs with consecutive requests
print("Possible Technician Pairs with Consecutive Requests:")
for tech_id, req1, req2 in tech_combis:
    print(f"Technician ID {tech_id}: Requests {req1} and {req2}")
    
tech_combis


Possible Technician Pairs with Consecutive Requests:
Technician ID 2: Requests 5 and 6
Technician ID 3: Requests 8 and 9
Technician ID 6: Requests 5 and 6


[(2, 5, 6), (3, 8, 9), (6, 5, 6)]

In [51]:
def weighted_tech_combis(tech_combis, machine_count):
    """
    Calculate the weight of each tech combi based on the machine count of the requests it contains.

    Args:
    - tech_combis: List of tuples where each tuple contains a tech ID and two request IDs.
    - machine_count: Dictionary where keys are request IDs and values are the machine count.

    Returns:
    - weighted_combis: Dictionary where keys are tech combis (tuples) and values are their weights.
    """
    weighted_combis = {}
    
    # Iterate through each tech combi
    for tech_id, req1, req2 in tech_combis:
        # Calculate the weight of the tech combi by summing the machine counts of the requests it contains
        weight = machine_count.get(req1, 0) + machine_count.get(req2, 0)
        weighted_combis[(tech_id, req1, req2)] = weight
    
    return weighted_combis

# Example usage:
weighted_combis = weighted_tech_combis(tech_combis, machine_count)

# Print weighted tech combis
print("Weighted Tech Combis:")
for combi, weight in weighted_combis.items():
    print(f"Tech Combi {combi}: Weight {weight}")

weighted_combis

Weighted Tech Combis:
Tech Combi (2, 5, 6): Weight 3
Tech Combi (3, 8, 9): Weight 4
Tech Combi (6, 5, 6): Weight 3


{(2, 5, 6): 3, (3, 8, 9): 4, (6, 5, 6): 3}

In [52]:
def feasible_tech_combi(weighted_combis, max_installations):
    """
    Check if each tech combi is feasible for a technician based on their maximum installations per day.

    Args:
    - weighted_combis: Dictionary with tech combis as keys and their weights as values.
    - max_installations: Dictionary where keys are technician IDs and values are their maximum installations.

    Returns:
    - feasible_combis: Dictionary with tech combis as keys and boolean values indicating feasibility.
    """
    feasible_combis = {}
    for tech_combi, combi_weight in weighted_combis.items():
        tech_id = tech_combi[0]
        max_installation = max_installations.get(tech_id, 0)
        feasible_combis[tech_combi] = combi_weight <= max_installation
    return feasible_combis


feasible_combis = feasible_tech_combi(weighted_combis, max_installations)

# Print the feasibility of each tech combi
for tech_combi, is_feasible in feasible_combis.items():
    print(f"Is tech combi {tech_combi} feasible? {is_feasible}")


Is tech combi (2, 5, 6) feasible? True
Is tech combi (3, 8, 9) feasible? True
Is tech combi (6, 5, 6) feasible? True


In [83]:
final_filtered_combinations_dict = {}

# Transform final_filtered_combinations into the desired format
for tech_id, days_combinations in final_filtered_combinations.items():
    for day, pairs_distances in days_combinations.items():
        for pair_distance in pairs_distances:
            if pair_distance:
                req1, req2 = pair_distance[0]
                final_filtered_combinations_dict[(tech_id, req1, req2)] = True
print(final_filtered_combinations_dict)

{(2, 5, 6): True, (2, 5, 10): True, (2, 6, 10): True, (2, 26, 28): True, (2, 26, 29): True, (2, 28, 29): True, (3, 1, 2): True, (3, 1, 4): True, (3, 2, 4): True, (3, 8, 9): True, (3, 11, 13): True, (3, 11, 15): True, (3, 13, 15): True, (3, 18, 19): True, (3, 18, 20): True, (3, 18, 21): True, (3, 20, 21): True, (4, 1, 2): True, (4, 16, 21): True, (4, 20, 21): True, (6, 5, 6): True, (6, 5, 7): True, (6, 6, 7): True, (8, 27, 30): True}


In [102]:
# read again, omdat bij de tech assignments de request array weg gaat
"""
Enter filepath of instance and desired name for the solution file
"""
directory = "/Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project"
filename = filename
output_file_name = output_file_name


"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}

print("Machines:")
print(machines)

print("\nLocations:")
print(locations)

print("\nRequests:")
print(requests)

print("\nTechnicians:")
print(technicians)


print(DAYS)
print(TRUCK_CAPACITY)
print(TRUCK_MAX_DISTANCE)

print(TRUCK_DISTANCE_COST)
print(TRUCK_DAY_COST)
print(TRUCK_COST)
print(TECHNICIAN_DISTANCE_COST)
print(TECHNICIAN_DAY_COST)
print(TECHNICIAN_COST)



Machines:
[[  1   6 155]
 [  2   3 421]
 [  3   7 593]]

Locations:
[[  1 675 563]
 [  2 742 453]
 [  3 325 514]
 [  4 595 438]
 [  5 583 353]
 [  6 316 466]
 [  7 608 353]
 [  8 326 473]
 [  9 533 437]
 [ 10 689 690]
 [ 11 551 677]
 [ 12 571 620]
 [ 13 411 373]
 [ 14 742 455]
 [ 15 543 360]
 [ 16 237 349]]

Requests:
[[ 1  8  1  2  3  1]
 [ 2  3  1  4  3  2]
 [ 3 11  1  4  2  2]
 [ 4  4  1  4  3  1]
 [ 5 12  2  3  2  1]
 [ 6 12  2  3  2  2]
 [ 7 11  2  4  3  1]
 [ 8  2  2  5  3  2]
 [ 9  2  2  3  3  2]
 [10 10  2  4  2  2]
 [11  4  3  4  3  1]
 [12 12  3  6  1  1]
 [13 15  3  5  3  1]
 [14  9  3  5  2  2]
 [15 14  3  4  3  1]
 [16 15  4  7  1  2]
 [17  3  4  5  2  1]
 [18  4  4  5  3  1]
 [19  2  4  6  3  2]
 [20  8  4  7  3  2]
 [21  6  4  7  3  1]
 [22  5  5  7  3  2]
 [23  4  5  8  2  3]
 [24 13  5  6  2  3]
 [25  5  6  7  3  2]
 [26  4  6  7  2  1]
 [27  3  6  8  2  2]
 [28 12  6  8  2  1]
 [29 14  6  9  2  1]
 [30  6  6  9  1  1]]

Technicians:
[[  1   7 212   5   0   1   0]
 [  

In [104]:
def time_window_for_requests(requests):
    time_windows = {}
    for request_info in requests:
        request_id = request_info[0]
        start_day = request_info[2]
        end_day = request_info[3]
        duration = end_day - start_day + 1  # Calculate duration in days
        time_windows[request_id] = duration
    return time_windows


time_windows = time_window_for_requests(requests)
print(time_windows)


{1: 2, 2: 4, 3: 4, 4: 4, 5: 2, 6: 2, 7: 3, 8: 4, 9: 2, 10: 3, 11: 2, 12: 4, 13: 3, 14: 3, 15: 2, 16: 4, 17: 2, 18: 2, 19: 3, 20: 4, 21: 4, 22: 3, 23: 4, 24: 2, 25: 2, 26: 2, 27: 3, 28: 3, 29: 4, 30: 4}


In [53]:
def technician_rare_request_count(double_check_requests, rare_requests):
    # Initialize a dictionary to store the rare request count for each technician
    technician_rare_counts = {tech_id: 0 for tech_id in double_check_requests.keys()}
    
    # Iterate through rare requests and update the count for each technician
    for req_id, data in rare_requests.items():
        technicians = data['technicians']
        for tech_id in technicians:
            technician_rare_counts[tech_id] += 1
    
    return technician_rare_counts

In [105]:
# Function to reset consecutive working days for a technician after rest days
def reset_consecutive_days(tech_id, rest_days):
    consecutive_days[tech_id] = 0
    last_day_worked[tech_id] += rest_days

# Function to handle rest days for a technician
def handle_rest_days(tech_id, rest_days):
    reset_consecutive_days(tech_id, rest_days)


# Function to attempt to assign a request to a technician
def try_assign_request(request_id, day_to_assign, available_technicians):
    min_rare_count = min(technician_rare_requests[tech_id] for tech_id in available_technicians)
    min_consecutive_days = min(consecutive_days[tech_id] for tech_id in available_technicians if technician_rare_requests[tech_id] == min_rare_count)
    suitable_technicians = [tech_id for tech_id in available_technicians if technician_rare_requests[tech_id] == min_rare_count and consecutive_days[tech_id] == min_consecutive_days]

    if not suitable_technicians:
        return False

    # Sort available technicians based on the smallest time window of their assigned requests
    available_technicians.sort(key=lambda tech_id: time_windows.get(tech_id, float('inf')))

    min_assigned = min(suitable_technicians, key=lambda x: (technician_rare_requests[x], consecutive_days[x]))

    if consecutive_days[min_assigned] >= 4:
        rest_days = 1 if consecutive_days[min_assigned] == 4 else 2
        handle_rest_days(min_assigned, rest_days)

    if any(day == day_to_assign for _, day in assigned_requests.get(min_assigned, [])):
        suitable_technicians.remove(min_assigned)
        if not suitable_technicians:
            return False
        min_assigned = min(suitable_technicians, key=lambda x: (technician_rare_requests[x], consecutive_days[x]))

    assigned_requests.setdefault(min_assigned, []).append((request_id, day_to_assign))
    assigned_request_ids.add(request_id)

    if last_day_worked[min_assigned] is not None and day_to_assign == last_day_worked[min_assigned] + 1:
        consecutive_days[min_assigned] += 1
    else:
        consecutive_days[min_assigned] = 1
    last_day_worked[min_assigned] = day_to_assign

    if day_to_assign > assignment_day:
        later_deliveries[request_id] = day_to_assign - assignment_day

    combi_assigned = False
    if min_assigned in feasible_combis_dict:
        for req1, req2 in feasible_combis_dict[min_assigned]:
            if request_id == req1 and req2 not in assigned_request_ids:
                assigned_requests[min_assigned].append((req2, day_to_assign))
                assigned_request_ids.add(req2)
                combi_assigned = True
                break
            elif request_id == req2 and req1 not in assigned_request_ids:
                assigned_requests[min_assigned].append((req1, day_to_assign))
                assigned_request_ids.add(req1)
                combi_assigned = True
                break

        if combi_assigned:
            feasible_combis_dict[min_assigned] = [(req1, req2) for req1, req2 in feasible_combis_dict[min_assigned]
                                                  if req1 not in assigned_request_ids and req2 not in assigned_request_ids]

    return True


    # Iterate through each request in the assignment schedule
    for request_id, assignment_day in sorted(assignment_schedule.items(), key=lambda x: x[1]):
        if request_id in assigned_request_ids:
            continue
        
        assigned = False
        for day_delay in range(days + 1):
            day_to_assign = assignment_day + day_delay
            if day_to_assign > days:
                unassigned_requests.add(request_id)
                break

            available_technicians = [tech_id for tech_id, requests in double_check_requests.items() if request_id in requests]

            if not available_technicians:
                continue

            if try_assign_request(request_id, day_to_assign, available_technicians):
                assigned = True
                break

        if not assigned:
            if assignment_day <= days:
                unassigned_requests.add(request_id)

    # Second pass: try to reassign unassigned requests
    remaining_unassigned_requests = list(unassigned_requests)
    unassigned_requests.clear()
    for request_id in remaining_unassigned_requests:
        assigned = False
        for day_to_assign in range(1, days + 1):
            available_technicians = [tech_id for tech_id, requests in double_check_requests.items() if request_id in requests]

            if not available_technicians:
                continue

            if try_assign_request(request_id, day_to_assign, available_technicians):
                assigned = True
                break

        if not assigned:
            unassigned_requests.add(request_id)

    return assigned_requests, later_deliveries, unassigned_requests

# First, calculate the rare request counts for each technician
technician_rare_requests = technician_rare_request_count(double_check_requests, rare_requests_dict)

# Then, call the assign_requests function with the calculated rare request counts
assigned_requests, later_deliveries, unassigned_requests = assign_requests(assignment_schedule, double_check_requests, DAYS, feasible_combis, technician_rare_requests)

# Print the assigned requests
for technician, requests in assigned_requests.items():
    print(f"Technician {technician} assigned requests:")
    for request, day in requests:
        print(f"Request {request} assigned on day {day}")

# Print the later deliveries
print("\nLater deliveries:")
for request_id, delay in later_deliveries.items():
    print(f"Request {request_id} assigned {delay} day(s) later")

# Print the unassigned requests
print("\nUnassigned requests:")
for request_id in unassigned_requests:
    print(f"Request {request_id} could not be assigned within {DAYS} days")

Technician 4 assigned requests:
Request 1 assigned on day 2
Request 2 assigned on day 2
Request 16 assigned on day 5
Request 21 assigned on day 5
Request 20 assigned on day 6
Technician 6 assigned requests:
Request 3 assigned on day 2
Request 5 assigned on day 3
Request 6 assigned on day 3
Request 12 assigned on day 4
Request 28 assigned on day 7
Technician 3 assigned requests:
Request 4 assigned on day 2
Request 8 assigned on day 3
Request 9 assigned on day 3
Request 11 assigned on day 4
Request 13 assigned on day 4
Request 15 assigned on day 5
Request 18 assigned on day 6
Request 19 assigned on day 6
Technician 10 assigned requests:
Request 7 assigned on day 3
Technician 2 assigned requests:
Request 10 assigned on day 3
Request 14 assigned on day 4
Request 29 assigned on day 7
Technician 8 assigned requests:
Request 17 assigned on day 5
Request 24 assigned on day 6
Request 27 assigned on day 7
Request 30 assigned on day 7
Technician 1 assigned requests:
Request 23 assigned on day 6
R

In [81]:
print(feasible_combis)
print(final_filtered_combinations)
print(final_filtered_combinations_dict)

{(2, 5, 6): True, (3, 8, 9): True, (6, 5, 6): True}
{1: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []}, 2: {2: [], 3: [((5, 6), 238), ((5, 10), 385), ((6, 10), 385)], 4: [], 5: [], 6: [], 7: [((26, 28), 452), ((26, 29), 425), ((28, 29), 485)]}, 3: {2: [((1, 2), 340), ((1, 4), 600), ((2, 4), 643)], 3: [((8, 9), 682)], 4: [((11, 13), 423), ((11, 15), 686), ((13, 15), 696)], 5: [((18, 19), 685), ((18, 20), 600), ((18, 21), 610), ((20, 21), 278)], 6: [], 7: []}, 4: {2: [((1, 2), 104)], 3: [], 4: [], 5: [((16, 21), 502), ((20, 21), 26)], 6: [], 7: []}, 5: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []}, 6: {2: [], 3: [((5, 6), 0), ((5, 7), 122), ((6, 7), 122)], 4: [], 5: [], 6: [], 7: []}, 7: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []}, 8: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [((27, 30), 104)]}, 9: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []}, 10: {2: [], 3: [], 4: [], 5: [], 6: [], 7: []}}
{(2, 5, 6): True, (2, 5, 10): True, (2, 6, 10): True, (2, 26, 28): True, (2, 26, 29): True, (2, 28, 29): True, 

In [80]:
final_filtered_combinations_dict = {}

# Transform final_filtered_combinations into the desired format
for tech_id, days_combinations in final_filtered_combinations.items():
    for day, pairs_distances in days_combinations.items():
        for pair_distance in pairs_distances:
            if pair_distance:
                req1, req2 = pair_distance[0]
                final_filtered_combinations_dict[(tech_id, req1, req2)] = True


In [ ]:
#### inn the code hierboven niet alleen de tech_combis doen maar juist de final_filtered_combinatins

In [55]:
def idle_request_days(later_deliveries):
    """
    Calculate the number of days each request is assigned later than its scheduled day.

    Args:
    - later_deliveries: Dictionary where keys are request IDs and values are the number of days delayed.

    Returns:
    - idle_days: Dictionary where keys are request IDs and values are the number of days each request is late.
    """
    idle_days = {}
    for request_id, delay in later_deliveries.items():
        idle_days[request_id] = delay

    return idle_days


# Example usage with the previous results
idle_days = idle_request_days(later_deliveries)

# Print idle request days
print("\nIdle request days:")
for request_id, days_late in idle_days.items():
    print(f"Request {request_id} is {days_late} day(s) too late")

idle_days



Idle request days:
Request 2 is 1 day(s) too late
Request 15 is 1 day(s) too late
Request 18 is 1 day(s) too late
Request 20 is 1 day(s) too late


{2: 1, 15: 1, 18: 1, 20: 1}

In [56]:
# read again, omdat bij de tech assignments de request array weg gaat
"""
Enter filepath of instance and desired name for the solution file
"""
directory = "/Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project"
filename = filename
output_file_name = output_file_name


"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}

print("Machines:")
print(machines)

print("\nLocations:")
print(locations)

print("\nRequests:")
print(requests)

print("\nTechnicians:")
print(technicians)


print(DAYS)
print(TRUCK_CAPACITY)
print(TRUCK_MAX_DISTANCE)

print(TRUCK_DISTANCE_COST)
print(TRUCK_DAY_COST)
print(TRUCK_COST)
print(TECHNICIAN_DISTANCE_COST)
print(TECHNICIAN_DAY_COST)
print(TECHNICIAN_COST)



Machines:
[[  1   6 155]
 [  2   3 421]
 [  3   7 593]]

Locations:
[[  1 675 563]
 [  2 742 453]
 [  3 325 514]
 [  4 595 438]
 [  5 583 353]
 [  6 316 466]
 [  7 608 353]
 [  8 326 473]
 [  9 533 437]
 [ 10 689 690]
 [ 11 551 677]
 [ 12 571 620]
 [ 13 411 373]
 [ 14 742 455]
 [ 15 543 360]
 [ 16 237 349]]

Requests:
[[ 1  8  1  2  3  1]
 [ 2  3  1  4  3  2]
 [ 3 11  1  4  2  2]
 [ 4  4  1  4  3  1]
 [ 5 12  2  3  2  1]
 [ 6 12  2  3  2  2]
 [ 7 11  2  4  3  1]
 [ 8  2  2  5  3  2]
 [ 9  2  2  3  3  2]
 [10 10  2  4  2  2]
 [11  4  3  4  3  1]
 [12 12  3  6  1  1]
 [13 15  3  5  3  1]
 [14  9  3  5  2  2]
 [15 14  3  4  3  1]
 [16 15  4  7  1  2]
 [17  3  4  5  2  1]
 [18  4  4  5  3  1]
 [19  2  4  6  3  2]
 [20  8  4  7  3  2]
 [21  6  4  7  3  1]
 [22  5  5  7  3  2]
 [23  4  5  8  2  3]
 [24 13  5  6  2  3]
 [25  5  6  7  3  2]
 [26  4  6  7  2  1]
 [27  3  6  8  2  2]
 [28 12  6  8  2  1]
 [29 14  6  9  2  1]
 [30  6  6  9  1  1]]

Technicians:
[[  1   7 212   5   0   1   0]
 [  

In [57]:
def get_idle_days_per_machine(idle_days, requests):
    """
    Calculate the total idle days for each machine type.

    Args:
    - idle_days: Dictionary where keys are request IDs and values are the number of days each request is late.
    - requests: List of requests where each request is a tuple.

    Returns:
    - machine_idle_days: Dictionary where keys are machine types and values are the total number of idle days.
    """
    # Get machine types from requests
    machine_types = get_machine_type_from_request(requests)
    
    # Initialize dictionary to store idle days for each machine type
    machine_idle_days = {}

    # Iterate through the idle days dictionary
    for request_id, days_late in idle_days.items():
        # Get the machine type for this request
        machine_type = machine_types.get(request_id)
        
        if machine_type:
            # Add the idle days to the corresponding machine type
            if machine_type in machine_idle_days:
                machine_idle_days[machine_type] += days_late
            else:
                machine_idle_days[machine_type] = days_late

    return machine_idle_days


machine_idle_days = get_idle_days_per_machine(idle_days, requests)

# Print the total idle days for each machine type
print("Total Idle Days for Each Machine Type:")
for machine_type, total_idle_days in machine_idle_days.items():
    print(f"{machine_type}: {total_idle_days} day(s)")


Total Idle Days for Each Machine Type:
3: 4 day(s)


In [58]:
def calculate_idle_costs(machine_idle_days, penalty_costs_machines):
    """
    Calculate the costs of idle machine days.

    Args:
    - machine_idle_days: Dictionary where keys are machine types and values are the total number of idle days.
    - penalty_costs_machines: Dictionary where keys are machine types and values are the penalty costs per day.

    Returns:
    - idle_costs: Dictionary where keys are machine types and values are the total costs of idle days.
    """
    idle_costs = {}
    for machine_type, idle_days in machine_idle_days.items():
        penalty_cost = penalty_costs_machines.get(machine_type, 0)
        idle_costs[machine_type] = idle_days * penalty_cost
    return idle_costs

# Example usage:
idle_costs = calculate_idle_costs(machine_idle_days, penalty_costs_machines)

# Print the total costs of idle days for each machine type
print("Total Costs of Idle Days for Each Machine Type:")
total_idle_costs = 0
for machine_type, total_costs in idle_costs.items():
    print(f"{machine_type}: {total_costs} cost units")
    total_idle_costs += total_costs

print(f"Total Idle Costs: {total_idle_costs} cost ")


Total Costs of Idle Days for Each Machine Type:
3: 2372 cost units
Total Idle Costs: 2372 cost 


In [59]:
def format_assigned_requests(assigned_requests, current_day=None):
    formatted_output = ""
    
    # Group requests by day
    requests_by_day = {}
    for technician, requests in assigned_requests.items():
        for request, day in requests:
            requests_by_day.setdefault(day, []).append((technician, request))
    
    # If current_day is provided, filter to only that day
    if current_day is not None:
        sorted_days = [current_day]
    else:
        sorted_days = sorted(requests_by_day.keys())
    
    # Iterate through each day
    for day in sorted_days:
        formatted_output += f"DAY {day}\n"
        requests_on_day = requests_by_day.get(day, [])
        
        # Count number of technicians
        num_technicians = len(set(technician for technician, _ in requests_on_day))
        formatted_output += f"NUMBER_OF_TECHNICIANS = {num_technicians}\n"
        
        # Group requests by technician
        requests_by_technician = {}
        for technician, request in requests_on_day:
            requests_by_technician.setdefault(technician, []).append(request)
        
        # Format requests for each technician
        for technician, requests in requests_by_technician.items():
            formatted_output += f"{technician} {' '.join(str(request) for request in requests)}\n"
        
        formatted_output += "\n"  # Add a newline between days
    
    return formatted_output

# Example usage
formatted_output = format_assigned_requests(assigned_requests)
print(formatted_output)

DAY 2
NUMBER_OF_TECHNICIANS = 3
4 1
6 3
3 4

DAY 3
NUMBER_OF_TECHNICIANS = 5
4 2
6 5 6
3 8 9
10 7
2 10

DAY 4
NUMBER_OF_TECHNICIANS = 4
4 13
6 12
3 11
2 14

DAY 5
NUMBER_OF_TECHNICIANS = 3
4 16
3 15
8 17

DAY 6
NUMBER_OF_TECHNICIANS = 4
4 20
3 18
8 24
1 23

DAY 7
NUMBER_OF_TECHNICIANS = 5
6 28
2 29
8 27
1 26
9 30




In [60]:
def store_technician_routes(assigned_requests):
    technician_routes = {}
    
    # Group requests by technician
    for technician, requests in assigned_requests.items():
        # Sort requests by day
        sorted_requests = sorted(requests, key=lambda x: x[1])
        
        # Initialize the route for the technician
        technician_routes[technician] = [[] for _ in range(max(day for _, day in sorted_requests) + 1)]
        
        # Fill in the route
        for request, day in sorted_requests:
            technician_routes[technician][day].append(request)
    
    return technician_routes

# Example usage
technician_routes = store_technician_routes(assigned_requests)

# Print the technician routes
for technician, route in technician_routes.items():
    print(f"Technician {technician} route:")
    for day, requests in enumerate(route):
        if requests:  # Check if the array is non-empty
            print(f"DAY {day}: {requests}")
    print()


Technician 4 route:
DAY 2: [1]
DAY 3: [2]
DAY 4: [13]
DAY 5: [16]
DAY 6: [20]

Technician 6 route:
DAY 2: [3]
DAY 3: [5, 6]
DAY 4: [12]
DAY 7: [28]

Technician 3 route:
DAY 2: [4]
DAY 3: [8, 9]
DAY 4: [11]
DAY 5: [15]
DAY 6: [18]

Technician 10 route:
DAY 3: [7]

Technician 2 route:
DAY 3: [10]
DAY 4: [14]
DAY 7: [29]

Technician 8 route:
DAY 5: [17]
DAY 6: [24]
DAY 7: [27]

Technician 1 route:
DAY 6: [23]
DAY 7: [26]

Technician 9 route:
DAY 7: [30]



In [61]:
def store_all_technician_routes(technician_routes):
    all_tech_routes = []
    for technician, route in technician_routes.items():
        for requests in route:
            all_tech_routes.extend([(technician, request) for request in requests])
    return all_tech_routes

# Example usage
all_tech_routes = store_all_technician_routes(technician_routes)

# Print all technician routes
print("All technician routes:")
print(all_tech_routes)


All technician routes:
[(4, 1), (4, 2), (4, 13), (4, 16), (4, 20), (6, 3), (6, 5), (6, 6), (6, 12), (6, 28), (3, 4), (3, 8), (3, 9), (3, 11), (3, 15), (3, 18), (10, 7), (2, 10), (2, 14), (2, 29), (8, 17), (8, 24), (8, 27), (1, 23), (1, 26), (9, 30)]


In [62]:
def calculate_distances_for_all_technician_routes(all_tech_routes, distances):
    tech_route_distances = {}
    for tech, req in all_tech_routes:
        distance = distances[(tech, req)]
        if tech not in tech_route_distances:
            tech_route_distances[tech] = 0
        tech_route_distances[tech] += distance
    return tech_route_distances

# Example usage
tech_route_distances = calculate_distances_for_all_technician_routes(all_tech_routes, distances)

# Print technician route distances
print("Technician route distances:")
for technician, distance in tech_route_distances.items():
    print(f"Technician {technician} total distance: {distance}")
    
def calculate_total_distance_for_all_technicians(tech_route_distances):
    total_distance = sum(tech_route_distances.values())
    return total_distance

# Example usage
total_distance_all_techs = calculate_total_distance_for_all_technicians(tech_route_distances)

# Print the total distance for all technicians
print(f"Total distance for all technicians: {total_distance_all_techs}")



Technician route distances:
Technician 4 total distance: 577
Technician 6 total distance: 61
Technician 3 total distance: 1612
Technician 10 total distance: 0
Technician 2 total distance: 446
Technician 8 total distance: 216
Technician 1 total distance: 172
Technician 9 total distance: 142
Total distance for all technicians: 3226


In [63]:
def print_schedule_for_each_day(truck_routes, delivery_days):
    all_days = sorted(set(delivery_days.values()))  # Get all unique delivery days
    
    for current_day in all_days:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        
        if num_trucks_for_day > 0:
            print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
            for truck_index, route in enumerate(truck_routes_for_day, start=1):
                print(truck_index, end=' ')
                for request_id in route:
                    print(request_id, end=' ')  # Print each request ID in the route
                print()  # Newline after each truck route
        else:
            print("NUMBER_OF_TRUCKS = 0")

# Example usage
print("Truck Routes with Depot:")
print_schedule_for_each_day(truck_routes_extracted, delivery_days)


Truck Routes with Depot:

DAY = 1
NUMBER_OF_TRUCKS = 3
1 1 
2 2 
3 3 4 

DAY = 2
NUMBER_OF_TRUCKS = 5
1 5 6 
2 7 
3 8 
4 9 
5 10 

DAY = 3
NUMBER_OF_TRUCKS = 3
1 11 12 
2 13 
3 14 15 

DAY = 4
NUMBER_OF_TRUCKS = 6
1 16 
2 17 
3 18 
4 19 
5 20 
6 21 

DAY = 5
NUMBER_OF_TRUCKS = 3
1 22 
2 23 
3 24 

DAY = 6
NUMBER_OF_TRUCKS = 5
1 25 
2 26 
3 27 28 
4 29 
5 30 


In [64]:
def print_schedule(truck_routes, delivery_days, assigned_requests, title, total_truck_distance, total_distance_all_techs):
    IDLE_MACHINE_COST = 0         # Define the actual cost
    
    # Calculate additional statistics
    days_with_trucks = set(day for _, day in truck_routes)
    days_with_technicians = set(day for day in delivery_days.values())
    all_days = sorted(set(days_with_trucks) | set(days_with_technicians))  # Union of days with trucks and technician days
    
    max_trucks_used = max(sum(1 for route, day in truck_routes if day == current_day) for current_day in days_with_trucks)
    max_technicians_used = max(sum(1 for day in delivery_days.values() if day == current_day) for current_day in days_with_technicians)
    
    number_of_truck_days = len(days_with_trucks)
    number_of_technician_days = len(days_with_technicians)

    # Calculate costs
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    # Print the additional information
    print("DATASET = ORTEC Caroline VeRoLog 2019")
    print(f"NAME = {title}")
    print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
    print(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}")
    print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
    print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
    print(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}")
    print(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}")
    print(f"IDLE_MACHINE_COSTS = {IDLE_MACHINE_COST}")
    print(f"TOTAL_COST = {int(total_cost)}")
    print()

    # Print the schedule for each day
    for current_day in all_days:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            print(truck_index, end=' ')
            for request_id in route:
                print(request_id, end=' ')  # Print each request ID in the route
            print()  # Newline after each truck route
        
        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        print(formatted_output)

# Example usage
print_schedule(truck_routes_extracted, delivery_days, assigned_requests, title, total_truck_distance, total_distance_all_techs)


DATASET = ORTEC Caroline VeRoLog 2019
NAME = Instance 12
TRUCK_DISTANCE = 11903
NUMBER_OF_TRUCK_DAYS = 6
NUMBER_OF_TRUCKS_USED = 6
TECHNICIAN_DISTANCE = 3226
NUMBER_OF_TECHNICIAN_DAYS = 6
NUMBER_OF_TECHNICIANS_USED = 6
IDLE_MACHINE_COSTS = 0
TOTAL_COST = 119954960


DAY = 1
NUMBER_OF_TRUCKS = 3
1 1 
2 2 
3 3 4 
NUMBER_OF_TECHNICIANS = 0



DAY = 2
NUMBER_OF_TRUCKS = 5
1 5 6 
2 7 
3 8 
4 9 
5 10 
NUMBER_OF_TECHNICIANS = 3
4 1
6 3
3 4



DAY = 3
NUMBER_OF_TRUCKS = 3
1 11 12 
2 13 
3 14 15 
NUMBER_OF_TECHNICIANS = 5
4 2
6 5 6
3 8 9
10 7
2 10



DAY = 4
NUMBER_OF_TRUCKS = 6
1 16 
2 17 
3 18 
4 19 
5 20 
6 21 
NUMBER_OF_TECHNICIANS = 4
4 13
6 12
3 11
2 14



DAY = 5
NUMBER_OF_TRUCKS = 3
1 22 
2 23 
3 24 
NUMBER_OF_TECHNICIANS = 3
4 16
3 15
8 17



DAY = 6
NUMBER_OF_TRUCKS = 5
1 25 
2 26 
3 27 28 
4 29 
5 30 
NUMBER_OF_TECHNICIANS = 4
4 20
3 18
8 24
1 23




In [65]:
print(title)

Instance 12


In [66]:
def print_schedule(truck_routes, delivery_days, assigned_requests):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks
    
    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)
    
    all_days = sorted(all_days)  # Sort all unique days
    
    for current_day in all_days:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            print(truck_index, end=' ')
            for request_id in route:
                print(request_id, end=' ')  # Print each request ID in the route
            print()  # Newline after each truck route
        
        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        print(formatted_output)

# Example usage
print_schedule(truck_routes_extracted, delivery_days, assigned_requests)



DAY = 1
NUMBER_OF_TRUCKS = 3
1 1 
2 2 
3 3 4 
NUMBER_OF_TECHNICIANS = 0



DAY = 2
NUMBER_OF_TRUCKS = 5
1 5 6 
2 7 
3 8 
4 9 
5 10 
NUMBER_OF_TECHNICIANS = 3
4 1
6 3
3 4



DAY = 3
NUMBER_OF_TRUCKS = 3
1 11 12 
2 13 
3 14 15 
NUMBER_OF_TECHNICIANS = 5
4 2
6 5 6
3 8 9
10 7
2 10



DAY = 4
NUMBER_OF_TRUCKS = 6
1 16 
2 17 
3 18 
4 19 
5 20 
6 21 
NUMBER_OF_TECHNICIANS = 4
4 13
6 12
3 11
2 14



DAY = 5
NUMBER_OF_TRUCKS = 3
1 22 
2 23 
3 24 
NUMBER_OF_TECHNICIANS = 3
4 16
3 15
8 17



DAY = 6
NUMBER_OF_TRUCKS = 5
1 25 
2 26 
3 27 28 
4 29 
5 30 
NUMBER_OF_TECHNICIANS = 4
4 20
3 18
8 24
1 23



DAY = 7
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 5
6 28
2 29
8 27
1 26
9 30




In [67]:
def print_schedule(truck_routes, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks
    max_trucks_used = 0
    max_technicians_used = 0
    number_of_truck_days = 0
    number_of_technician_days = 0

    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)

    all_days = sorted(all_days)  # Sort all unique days

    for current_day in all_days:
        num_trucks_for_day = 0
        num_technicians_for_day = 0

        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")

        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            number_of_truck_days += 1
        if num_trucks_for_day > max_trucks_used:
            max_trucks_used = num_trucks_for_day
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            print(truck_index, end=' ')
            for request_id in route:
                print(request_id, end=' ')  # Print each request ID in the route
            print()  # Newline after each truck route

        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        num_technicians_for_day = formatted_output.count('NUMBER_OF_TECHNICIANS = ')
        if num_technicians_for_day > 0:
            technician_info = formatted_output.split('NUMBER_OF_TECHNICIANS = ')[1]
            num_technicians = int(technician_info.split()[0])
            if num_technicians > 0:
                number_of_technician_days += 1
                if num_technicians > max_technicians_used:
                    max_technicians_used = num_technicians
        if num_technicians_for_day > max_technicians_used:
            max_technicians_used = num_technicians_for_day
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        print(formatted_output)

    
    IDLE_MACHINE_COST = total_idle_costs

    
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    return (number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost)


# Example usage
number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost = print_schedule(truck_routes_extracted, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs)

# Now you have access to these calculated values outside the function scope
print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
print(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}")
print(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}")
print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
print(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}")
print(f"TOTAL_COST = {int(total_cost)}")



DAY = 1
NUMBER_OF_TRUCKS = 3
1 1 
2 2 
3 3 4 
NUMBER_OF_TECHNICIANS = 0



DAY = 2
NUMBER_OF_TRUCKS = 5
1 5 6 
2 7 
3 8 
4 9 
5 10 
NUMBER_OF_TECHNICIANS = 3
4 1
6 3
3 4



DAY = 3
NUMBER_OF_TRUCKS = 3
1 11 12 
2 13 
3 14 15 
NUMBER_OF_TECHNICIANS = 5
4 2
6 5 6
3 8 9
10 7
2 10



DAY = 4
NUMBER_OF_TRUCKS = 6
1 16 
2 17 
3 18 
4 19 
5 20 
6 21 
NUMBER_OF_TECHNICIANS = 4
4 13
6 12
3 11
2 14



DAY = 5
NUMBER_OF_TRUCKS = 3
1 22 
2 23 
3 24 
NUMBER_OF_TECHNICIANS = 3
4 16
3 15
8 17



DAY = 6
NUMBER_OF_TRUCKS = 5
1 25 
2 26 
3 27 28 
4 29 
5 30 
NUMBER_OF_TECHNICIANS = 4
4 20
3 18
8 24
1 23



DAY = 7
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 5
6 28
2 29
8 27
1 26
9 30


TRUCK_DISTANCE = 11903
NUMBER_OF_TRUCK_DAYS = 6
NUMBER_OF_TECHNICIAN_DAYS = 6
TECHNICIAN_DISTANCE = 3226
NUMBER_OF_TRUCKS_USED = 6
NUMBER_OF_TECHNICIANS_USED = 5
TOTAL_COST = 119957232


In [68]:
def final_print_schedule(title, truck_routes, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks
    max_trucks_used = 0
    max_technicians_used = 0
    number_of_truck_days = 0
    number_of_technician_days = 0

    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)

    all_days = sorted(all_days)  # Sort all unique days

    # Initialize formatted day-by-day output
    daily_output = []

    for current_day in all_days:
        num_trucks_for_day = 0
        num_technicians_for_day = 0

        day_output = []
        day_output.append(f"DAY = {current_day}")

        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            number_of_truck_days += 1
        if num_trucks_for_day > max_trucks_used:
            max_trucks_used = num_trucks_for_day
        day_output.append(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            route_output = f"{truck_index} " + ' '.join(map(str, route))
            day_output.append(route_output)

        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        num_technicians_for_day = formatted_output.count('NUMBER_OF_TECHNICIANS = ')
        if num_technicians_for_day > 0:
            technician_info = formatted_output.split('NUMBER_OF_TECHNICIANS = ')[1]
            num_technicians = int(technician_info.split()[0])
            if num_technicians > 0:
                number_of_technician_days += 1
                if num_technicians > max_technicians_used:
                    max_technicians_used = num_technicians
        if num_technicians_for_day > max_technicians_used:
            max_technicians_used = num_technicians_for_day
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        day_output.append(formatted_output)

        daily_output.append('\n'.join(day_output))

    # Print summary information
    IDLE_MACHINE_COST = total_idle_costs
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    print(f"DATASET = ORTEC Caroline VeRoLog 2019")
    print(f"NAME = {title}")
    print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
    print(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}")
    print(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}")
    print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
    print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
    print(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}")
    print(f"IDLE_MACHINE_COSTS = {total_idle_costs}")
    print(f"TOTAL_COST = {int(total_cost)}")

    # Print the day-by-day output
    for output in daily_output:
        print()
        print(output)

    return (number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost)

# Example usage
number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost = final_print_schedule(title, truck_routes_extracted, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs)


DATASET = ORTEC Caroline VeRoLog 2019
NAME = Instance 12
TRUCK_DISTANCE = 11903
NUMBER_OF_TRUCK_DAYS = 6
NUMBER_OF_TECHNICIAN_DAYS = 6
TECHNICIAN_DISTANCE = 3226
NUMBER_OF_TRUCKS_USED = 6
NUMBER_OF_TECHNICIANS_USED = 5
IDLE_MACHINE_COSTS = 2372
TOTAL_COST = 119957232

DAY = 1
NUMBER_OF_TRUCKS = 3
1 1
2 2
3 3 4
NUMBER_OF_TECHNICIANS = 0



DAY = 2
NUMBER_OF_TRUCKS = 5
1 5 6
2 7
3 8
4 9
5 10
NUMBER_OF_TECHNICIANS = 3
4 1
6 3
3 4



DAY = 3
NUMBER_OF_TRUCKS = 3
1 11 12
2 13
3 14 15
NUMBER_OF_TECHNICIANS = 5
4 2
6 5 6
3 8 9
10 7
2 10



DAY = 4
NUMBER_OF_TRUCKS = 6
1 16
2 17
3 18
4 19
5 20
6 21
NUMBER_OF_TECHNICIANS = 4
4 13
6 12
3 11
2 14



DAY = 5
NUMBER_OF_TRUCKS = 3
1 22
2 23
3 24
NUMBER_OF_TECHNICIANS = 3
4 16
3 15
8 17



DAY = 6
NUMBER_OF_TRUCKS = 5
1 25
2 26
3 27 28
4 29
5 30
NUMBER_OF_TECHNICIANS = 4
4 20
3 18
8 24
1 23



DAY = 7
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 5
6 28
2 29
8 27
1 26
9 30




In [69]:
def final_print_schedule(title, truck_routes, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs, output_file_name):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks
    max_trucks_used = 0
    max_technicians_used = 0
    number_of_truck_days = 0
    number_of_technician_days = 0

    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)

    all_days = sorted(all_days)  # Sort all unique days

    # Initialize formatted day-by-day output
    daily_output = []

    for current_day in all_days:
        num_trucks_for_day = 0
        num_technicians_for_day = 0

        day_output = []
        day_output.append(f"DAY = {current_day}")

        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            number_of_truck_days += 1
        if num_trucks_for_day > max_trucks_used:
            max_trucks_used = num_trucks_for_day
        day_output.append(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            route_output = f"{truck_index} " + ' '.join(map(str, route))
            day_output.append(route_output)

        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        num_technicians_for_day = formatted_output.count('NUMBER_OF_TECHNICIANS = ')
        if num_technicians_for_day > 0:
            technician_info = formatted_output.split('NUMBER_OF_TECHNICIANS = ')[1]
            num_technicians = int(technician_info.split()[0])
            if num_technicians > 0:
                number_of_technician_days += 1
                if num_technicians > max_technicians_used:
                    max_technicians_used = num_technicians
        if num_technicians_for_day > max_technicians_used:
            max_technicians_used = num_technicians_for_day
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        day_output.append(formatted_output)

        daily_output.append('\n'.join(day_output))

    # Calculate costs
    IDLE_MACHINE_COST = total_idle_costs
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    # Open a file to write the output
    with open(output_file_name, "w") as file:
        # Write the summary information
        file.write(f"DATASET = ORTEC Caroline VeRoLog 2019\n")
        file.write(f"NAME = {title}\n")
        file.write(f"TRUCK_DISTANCE = {int(total_truck_distance)}\n")
        file.write(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}\n")
        file.write(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}\n")
        file.write(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}\n")
        file.write(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}\n")
        file.write(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}\n")
        file.write(f"IDLE_MACHINE_COSTS = {total_idle_costs}\n")
        file.write(f"TOTAL_COST = {int(total_cost)}\n")

        # Write the day-by-day output
        for output in daily_output:
            file.write("\n")
            file.write(output)
            file.write("\n")

    return (number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost)

# Example usage
number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost = final_print_schedule(title, truck_routes_extracted, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs, output_file_name)
